# Main GDP Data 

Brian Dew

@bd_econ

In [2]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
#nipa_series_codes(retrieve_table('T11500'))

### GDP Intro Chart and Text

In [9]:
df = nipa_df(retrieve_table('T10502')['Data'],
             ['A191RL'])['A191RL']
# Dates for latest quarters chart
a = df.iloc[-8:]
a['Quarter'] = [f'Q{i.quarter}\\\\{i.year}' if i.quarter == 1 
                else dtxt(i)['qtr3'] for i in a.index]
# From GDP Now (cover cases where no nowcast available)
if gdpdt < nowdt:
    next_qtr = rec.index[-1] + pd.DateOffset(months=3)
    label = (f'Q{next_qtr.quarter}\\\\{next_qtr.year}' 
             if next_qtr.quarter == 1 else dtxt(next_qtr)['qtr3'])
    frow = pd.DataFrame({'Quarter': label, 
                         'index': next_qtr}, index={'index': 4})
    rec= pd.concat([rec.reset_index(), frow]).set_index('index')
    rec.loc[next_qtr, 'A191RL'] = 0
rec['zero'] = 0
a.to_csv(data_dir/'gdp_rec2.csv', index_label='date')

In [10]:
df = nipa_df(retrieve_table('T70100')['Data'], ['A939RC', 'A939RX'])
df['value'] = (df['A939RX'] / df['A939RX'].iloc[-1]) * df['A939RC'].iloc[-1]
df.loc['1989':, 'value'].divide(1000).to_csv(data_dir / 'gdppc.csv', 
                                             index_label='date')
cd = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC'].iloc[-1]
rgdp = nipa_df(retrieve_table('T10106')['Data'], ['A191RX'])
rgdp_cd = rgdp / rgdp.iloc[-1] * cd

ltdate = dtxt(df.index[-1])['qtr2']
gdp_val = int(rgdp_cd.iloc[-1].values[0] / 1000)
gdp_prv_val = int(rgdp_cd.loc['2019-10-01'] / 1000)
gdp_prv_val2 = int(rgdp_cd.iloc[0].values[0] / 1000)
gdppc_val = int(df.iloc[-1].loc['value'])
gdppc_prv_val = int(df.loc['2019-10-01', 'value'])
gdppc_prv_val2 = int(df.iloc[0].loc['value'])
qdate = dtxt(df.index[-1])['qtr1']
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']
num = numbers[f'{(pop.pct_change(4).iloc[-1] * 1000).round()}']
popnum = f'{num}-tenths of' if num != 'five' else 'half'
color = 'red!95!black'
cl = c_line(color)
node = end_node(df['value'] / 1000, color, anchor='south', 
                dollar='thousands')
write_txt(text_dir / 'gdp_pc_node.txt', node)
text = (f'\${gdp_val:,} billion in {ltdate}, compared to an '+
        f'inflation-adjusted equivalent of \${gdp_prv_val:,} '+
        f'billion in 2019 Q4, and \${gdp_prv_val2:,} billion in '+
        'the first quarter of 1989.\n\nThe US population is growing '+
        f'by about {popnum} a percent per year. GDP per capita '+
        f'{cl}, adjusted for inflation to {qdate} dollars, had '+
        f'increased to \${gdppc_prv_val:,} in 2019 Q4 from '+
        f'\${gdppc_prv_val2:,} in 1989 Q1, and is currently '+
        f'\${gdppc_val:,}, as of {ltdate}.')
write_txt(text_dir / 'gdp1.txt', text)
print(text)

\$27,623 billion in the third quarter of 2023, compared to an inflation-adjusted equivalent of \$25,731 billion in 2019 Q4, and \$11,504 billion in the first quarter of 1989.

The US population is growing by about half a percent per year. GDP per capita (see {\color{red!95!black}\textbf{---}}), adjusted for inflation to 2023 Q3 dollars, had increased to \$77,690 in 2019 Q4 from \$47,111 in 1989 Q1, and is currently \$82,335, as of the third quarter of 2023.


### GDP components

In [11]:
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']

srs = {'A191RX': '\hspace{0.01mm} {\color{red!95!black}\\textbf{---}} Gross Domestic Product', 
       'DPCERX': '\hspace{1.0mm} {\color{yellow!45!orange}\\textbf{---}} Consumer Spending', 
       'A006RX': '\hspace{1.0mm} {\color{blue!70!black}\\textbf{---}} Gross Private Domestic Investment', 
       'A822RX': '\hspace{1.0mm} {\color{cyan!60!white}\\textbf{---}} Government Spending \& Investment',
       'A019RX': '\hspace{1.0mm} {\color{green!60!black}\\textbf{---}} Net Exports', 
       'A020RX': '\hspace{4.5mm} Exports', 'A021RX': '\hspace{4.5mm} Less: Imports'}
s = [key for key, value in srs.items()]
s2 = [i[:-1].replace('A02', 'B02') + 'C' for i in s]
df = nipa_df(retrieve_table('T10106')['Data'], s).sort_index()
df2 = nipa_df(retrieve_table('T10105')['Data'], s2).sort_index()

dfpop = df.div(pop, axis=0)
real_vals = df2.rename({i: i.replace('RC', 'RX').replace('B02', 'A02') 
                        for i in s2}, axis=1).iloc[-1]
data = ((dfpop / dfpop.iloc[-1]) * (real_vals / pop.iloc[-1])).loc['1989':]

sel_cols = ['DPCERX', 'A006RX', 'A019RX', 'A822RX']

data[sel_cols].to_csv(data_dir / 'gdp_pc_comp.csv', index_label='date')

pce = f'\${data.DPCERX.iloc[-1] * 1000:,.0f}'
inv = f'\${data.A006RX.iloc[-1] * 1000:,.0f}'
gov = f'\${data.A822RX.iloc[-1] * 1000:,.0f}'
exp = f'\${abs(data.A019RX.iloc[-1]) * 1000:,.0f}'
ltdt = dtxt(data.index[-1])['qtr1']
pce_diff = f'\${(data.DPCERX.iloc[-1] - data.DPCERX.iloc[0]) * 1000:,.0f}'

text = ('Much of the increase in GDP over the past '+
        '30 years comes from consumer spending. Consumer '+
        'spending (see {\color{yellow!45!orange}\\textbf{---}}) is '+
        f'equivalent to {pce} per person in {ltdt}, a price-adjusted '+
        f'increase of {pce_diff} since 1989.\n\nGross private domestic '+
        'investment (see {\color{blue!70!black}\\textbf{---}}) is '+
        f'equivalent to {inv} per person in {ltdt}. Government '+
        'spending and investment (see {\color{cyan!60!white}\\textbf{---}}) '+
        f'totals {gov} per person. The trade deficit, equivalent to {exp} per '+
        'person, is subtracted, to capture only domestic production '+
        '(see {\color{green!60!black}\\textbf{---}}).')

write_txt(text_dir / 'gdp_pc_comp.txt', text)
write_txt(text_dir / 'gdp_ltdt.txt', ltdt)
print(text)

res = data * 1000
lt = res.rename(srs, axis=1).iloc[-1]
lt.name = dtxt(lt.name)['qtr1']
pr = res.rename(srs, axis=1).iloc[-2]
pr.name = dtxt(pr.name)['qtr1']
pc = res.rename(srs, axis=1).loc['2019-10-01']
pc.name = dtxt(pc.name)['qtr1']
p00 = res.rename(srs, axis=1).loc['2000-01-01']
p00.name = dtxt(p00.name)['qtr1']
fi = res.rename(srs, axis=1).iloc[0]
fi.name = dtxt(fi.name)['qtr1']
table = pd.concat([lt, pr, pc, p00, fi], axis=1).applymap('{:,.0f}'.format)
table.iloc[0, 0] = f'\${table.iloc[0, 0]}'
table.to_csv(data_dir / 'gdppc_levels.tex', sep='&', 
             lineterminator='\\\ ', quotechar=' ')

Much of the increase in GDP over the past 30 years comes from consumer spending. Consumer spending (see {\color{yellow!45!orange}\textbf{---}}) is equivalent to \$55,840 per person in 2023 Q3, a price-adjusted increase of \$25,483 since 1989.

Gross private domestic investment (see {\color{blue!70!black}\textbf{---}}) is equivalent to \$14,603 per person in 2023 Q3. Government spending and investment (see {\color{cyan!60!white}\textbf{---}}) totals \$14,231 per person. The trade deficit, equivalent to \$2,339 per person, is subtracted, to capture only domestic production (see {\color{green!60!black}\textbf{---}}).


### Labor Share

In [4]:
# Compensation of employees divided by gross domestic income less 
# depreciation (net domestic income, as depreciation is income that
# does not go to people)
s = ['A261RC', 'A4002C', 'A262RC', 'W271RC']
df = nipa_df(retrieve_table('T11000')['Data'], s).sort_index()
df['NDI'] = (df['A4002C'] + df['W271RC'])
df['Share'] = (df['A4002C'] / df['NDI']) * 100

s2 = ['A442RC', 'W322RC']
df2 = nipa_df(retrieve_table('T11400')['Data'], s2).sort_index()
df2['NCI'] = df2[s2].dropna().sum(axis=1)
csh = (df2['A442RC'] / df2['NCI']).dropna().rolling(4).mean() * 100
csh.name = 'Corp_Share'

data = pd.concat([df['Share'].rolling(4).mean(), csh], axis=1).dropna()
data.loc['1989':].to_csv(data_dir / 'laborshare.csv', 
                                  index_label='date')

colors = {'Share': 'blue!50!cyan', 'Corp_Share': 'green!60!blue'}
adj = node_adj(data)
smax = data.dropna().iloc[-1].idxmax()
adj[smax] += 0.35
date = {n: None for n in colors.keys()}
date[smax] = 'q'

nodes = '\n'.join([end_node(data[n].dropna(), c, date=date[n], offset=adj[n]) 
                   for n, c in colors.items()])
write_txt(text_dir / 'laborshare_node.txt', nodes)

# Text
ltdt = dtxt(data.index[-1])['qtr1']
ltval = value_text(data.Share.iloc[-1], 'plain')
ltvalc = value_text(data.Corp_Share.iloc[-1], 'plain')
chlt = value_text(data.Share.diff(4).dropna().iloc[-1], ptype='pp')
cmax = value_text(data.Corp_Share.max(), 'plain')
cidxmax = dtxt(data.Corp_Share.idxmax())['qtr1']
cch = value_text(data.Corp_Share.iloc[-1] - data.Corp_Share.max(), 
                 'above_below', ptype='pp')

# Note on value of labor share
emp = pd.read_csv(data_dir / 'ces_raw.csv', index_col='date', 
                  parse_dates=True)['CES0000000001'] / 1_000
emp = emp.resample('QS').mean()
tot = ((df['NDI'].dropna()) * 0.01)
ltndi = tot.iloc[-1] 
ltemp = emp.loc[tot.index[-1]]
per_emp = (ltndi / ltemp).round(-1)
cl = {n: c_line(c) for n, c in colors.items()}
ctxt = ('For context, one percent of net domestic income translates to '+
        f'\${tot.iloc[-1] / 1000:.0f} billion per year, which is '+
        f'\${per_emp:,.0f} per worker. ')

text = (f'Over the year ending {ltdt}, labor receives {ltval} '+
        f"of net domestic income {cl['Share']}. Labor's share {chlt} "+
        f"over the past year. {ctxt}\n\n Labor's share in the corporate "+
        f"sector is {ltvalc} percent in {ltdt} {cl['Corp_Share']}. The corporate "+
        'sector has well-defined accounting, which is useful for this '+
        f'analysis. The corporate labor share is currently {cch} '+
        f'its 30-year high of {cmax} in {cidxmax}. ')
write_txt(text_dir / 'laborshare.txt', text)
print(text)

Over the year ending 2023 Q2, labor receives 68.7 percent of net domestic income (see {\color{blue!50!cyan}\textbf{---}}). Labor's share increased 0.9 percentage point over the past year. For context, one percent of net domestic income translates to \$204 billion per year, which is \$1,310 per worker. 

 Labor's share in the corporate sector is 73.8 percent percent in 2023 Q2 (see {\color{green!60!blue}\textbf{---}}). The corporate sector has well-defined accounting, which is useful for this analysis. The corporate labor share is currently 9.1 percentage points below its 30-year high of 83.0 percent in 1993 Q1. 


### Nominal GDP decomposed as Real GDP and Prices

In [2]:
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
df = nipa_df(retrieve_table('T10502')['Data'], ['A191RL'])
gdpgr = growth_rate(gdp)
df['PR'] = growth_rate(gdp) - df['A191RL']
df.loc['1989':].to_csv(data_dir / 'ngdp_gr.csv', index_label='date')
(gdp.loc['1989':] / 1_000_000).to_csv(data_dir / 'ngdp.csv', 
                                      index_label='date')

# Text 
ltdt = dtxt(gdpgr.index[-1])['qtr2']
ltch = value_text(gdpgr.iloc[-1], adj='annual')
prch = prval_comp(gdpgr)

ann19 = value_text(cagr(gdp.loc['2019-10-01':]), 'plain')
ann89 = value_text(cagr(gdp.loc['1989-01-01':'2019-10-01']), 
                   adj='annual')
    
text = (f'In {ltdt}, nominal GDP {ltch}, following {prch}. '+
        f'From 1989 to 2019 Q4, nominal GDP {ann89}. '+
        f'Since 2019 Q4, the annualized growth rate is {ann19}.')
write_txt(text_dir / 'ngdp_gr.txt', text)
print(text)

In the third quarter of 2023, nominal GDP increased at an annual rate of 8.5 percent, following increases of 3.8 percent in Q2 and 6.3 percent in Q1. From 1989 to 2019 Q4, nominal GDP increased at an annual rate of 4.6 percent. Since 2019 Q4, the annualized growth rate is 6.4 percent.


### Investment overview

In [21]:
#nipa_series_codes(retrieve_table('T10105'))

In [71]:
d = {'T10105': {'A007RC': 'PrGrossFixed', 'A008RC': 'PrGrossFixedNR', 
                'A011RC': 'PrGrossFixedRes', 'A191RC': 'GDP', 
                'A014RC': 'CIPI'},
     'T30100': {'A782RC': 'GovGross'}}

df = pd.DataFrame()
for t, s in d.items():
    i = nipa_df(retrieve_table(t)['Data'], s.keys()).rename(d, axis=1)
    for v, n in s.items():
        df[n] = i[v]
        
# Sum public and private gross investment
df['Gross'] = df['PrGrossFixed'] + df['GovGross']
df.to_csv(data_dir / 'grossinv_lvl.csv', index_label='date')

# Share of GDP
sh = df.divide(df.GDP, axis=0) * 100
sh.loc['1989':].to_csv(data_dir / 'grossinv_sh.csv', index_label='date')

# Text 
ltdt = dtxt(sh.index[-1])['qtr2']
ltval = f'\${df.Gross.iloc[-1] / 1_000_000:.1f} trillion'
ltsh = f'{sh.Gross.iloc[-1]:.1f} percent of GDP'
prsh = f'{sh.Gross.iloc[-5]:.1f} percent of GDP'
prdt = dtxt(sh.index[-5])['qtr1'] 
sh19 = f'{sh.loc["2019", "Gross"].mean():.1f} percent of GDP'
also = 'also ' if prsh == sh19 else ''
cl = c_line('black')

text = (f'In {ltdt}, annualized US \\textbf{{gross fixed investment}}, '+
        f'both public and private, totals {ltval}, or {ltsh} '+
        f'{cl}. Gross fixed investment is equivalent to {prsh} '+
        f'one year prior, in {prdt}, and {also}averages {sh19} '+
        'in 2019. ')
write_txt(text_dir / 'gross_inv_total.txt', text)
print(text, '\n')

# Node for total
node = end_node(sh.Gross, 'black', date='qs', offset=0.2)
write_txt(text_dir / 'gross_inv_total_node.txt', node)
# Nodes for latest values
cols = ['GovGross', 'PrGrossFixedRes', 'PrGrossFixedNR']
sdf = sh[cols].iloc[-1]
height = ((sdf.cumsum() - (sdf / 2) + 1.5)).to_dict()
val = sdf.to_dict()
dtp = dtxt(sh.index[-1] + pd.DateOffset(months=2))['datetime']
nodes = [f'\\absnode{{{{{dtp}}}}}{{{height[i]}}}{{\scriptsize {val[i]:.1f}}}' 
         for i in cols]
nodetext = '\n'.join(nodes)
write_txt(text_dir / 'gross_inv_nodes.txt', nodetext)

# Color boxes in text
govcb = c_box('violet!60!black').replace('see ', 'see')
rescb = c_box('blue!60!white').replace('see ', 'see')
nrcb = c_box('yellow!70!orange!90!white').replace('see ', 'see')

# Text 2
ltdt = dtxt(sh.index[-1])['qtr1']
st = sh.iloc[-1].to_frame()
st['sh'] = (st / st.loc['Gross']) * 100
st['sh_txt'] = st['sh'].apply(value_text, style='plain', digits=0)
st['gdp_txt'] = st.iloc[:,0].apply(value_text, style='plain')

text = (f'In {ltdt}, private nonresidential (business) fixed investment '+
        f'comprises {st.loc["PrGrossFixedNR", "sh_txt"]} of the '+
        f'total and translates to {st.loc["PrGrossFixedNR", "gdp_txt"]} '+
        f'of GDP {nrcb}. Private residential makes up '+
        f'{st.loc["PrGrossFixedRes", "sh_txt"]} of the total and '+
        f'{st.loc["PrGrossFixedRes", "gdp_txt"]} of GDP {rescb}. Public '+
        f'investment is {st.loc["GovGross", "sh_txt"]} of the total '+
        f'and {st.loc["GovGross", "gdp_txt"]} of GDP {govcb}.')
print(text)
write_txt(text_dir / 'gross_inv_sector.txt', text)

In the third quarter of 2023, annualized US \textbf{gross fixed investment}, both public and private, totals \$5.8 trillion, or 21.0 percent of GDP (see {\color{black}\textbf{---}}). Gross fixed investment is equivalent to 21.3 percent of GDP one year prior, in 2022 Q3, and averages 21.4 percent of GDP in 2019.  

In 2023 Q3, private nonresidential (business) fixed investment comprises 64 percent of the total and translates to 13.5 percent of GDP (see\cbox{yellow!70!orange!90!white}). Private residential makes up 19 percent of the total and 3.9 percent of GDP (see\cbox{blue!60!white}). Public investment is 17 percent of the total and 3.6 percent of GDP (see\cbox{violet!60!black}).


In [24]:
#nipa_series_codes(retrieve_table('T10502'))

### Gross fixed investment

In [26]:
# Retrieve levels data 
lvl = pd.read_csv(data_dir / 'grossinv_lvl.csv', 
                  index_col='date', parse_dates=True)
# Contribution to one-year growth
ac = growth_contrib_ann(lvl, 'GDP')
act = value_text(ac.Gross.iloc[-1], 'contribution_to', 
                 ptype='pp', digits=2)

# Contribution to annualized quarterly growth
s = ['A008RY', 'A011RY', 'A014RY', 'A788RY', 'A798RY', 
     'A799RY', 'A007RY']
gc = nipa_df(retrieve_table('T10502')['Data'], s)
gc['Gov'] = gc[['A788RY', 'A798RY', 'A799RY']].sum(axis=1)
gc['Total'] = gc[['A007RY', 'Gov']].sum(axis=1)
gc.loc['1989':].to_csv(data_dir / 'inv.csv', index_label='date')

# Example of strong investment growth in late 1990s
ex = value_text(gc.loc['1996': '1999', 'Total'].mean(), 
                    'added_lost', adj='average', digits=2, ptype='pp')

# Generate text
ltdate = dtxt(gc.index[-1])['qtr2']
ltdt = dtxt(gc.index[-1])['qtr1']
prdt = dtxt(gc.index[-2])['qtr1']
prdate = dtxt(gc.index[-2])['qtr5']
pr_dt = prdate if gc.index[-1].year == gc.index[-2].year else prdt
tot = value_text(gc.Total.iloc[-1], 'contribution_to', ptype='pp', 
                 digits=2, threshold=0.01)
totpr = value_text(gc.Total.iloc[-2], 'contribution_of', ptype='point',
                 digits=2, threshold=0.01)

bus = value_text(gc.A008RY.iloc[-1], 'contribution_to', ptype='pp', 
                 digits=2, threshold=0.01)
res = value_text(gc.A011RY.iloc[-1], 'contribution', ptype='pp', 
                 digits=2, threshold=0.01)
pub = value_text(gc.Gov.iloc[-1], 'contribution', ptype='pp', 
                 digits=2, threshold=0.01)
cipi = value_text(gc.A014RY.iloc[-1], 'contribution_to', ptype='pp', 
                  digits=2, threshold=0.01)
cipipr = value_text(gc.A014RY.iloc[-2], 'contribution_of', ptype='pp', 
                  digits=2, threshold=0.01)
cipi_yr = value_text(ac.CIPI.iloc[-1], 'contribution', ptype='pp', 
                     digits=2, threshold=0.01)
nr_yr = value_text(ac.PrGrossFixedNR.iloc[-1], 'contribution', ptype='pp', 
                     digits=2, threshold=0.01)
res_yr = value_text(ac.PrGrossFixedRes.iloc[-1], 'contribution', 
                    ptype='point', digits=2, threshold=0.01)
pub_yr = value_text(ac.GovGross.iloc[-1], 'contribution', ptype='point', 
                     digits=2, threshold=0.01)

# Color boxes in text
cbpub = c_box('violet!60!black').replace('see ', 'see')
cbres = c_box('blue!60!white').replace('see ', 'see')
cbbus = c_box('yellow!70!orange!90!white').replace('see ', 'see')
cbcipi = c_box('red').replace('see ', 'see')

text = ('For example, from 1996 to 1999, gross domestic fixed '+
        f'investment {ex} to annual real GDP growth. \n\n'
        f'In {ltdate}, gross domestic fixed investment {tot} '+
        f'annualized real GDP growth, following {totpr} in {pr_dt}. '+
        f'Over the past year, gross fixed investment {act} real '+
        'GDP growth.\n\n'+
        f'In {ltdt}, by type of gross fixed investment, private '+
        f'nonresidential {bus} annualized real GDP growth {cbbus}, '+
        f'private residential {res} {cbres}, and public {pub} '+
        f'{cbpub}. Over the past year, nonresidential or business '+
        f'gross fixed investment {nr_yr}, residential {res_yr}, and '+
        f'public {pub_yr}.')
write_txt(text_dir / 'inv_contrib_text.txt', text)
print(text, '\n')

text = (f'In {ltdt}, changes in private inventories {cipi} '+
        f'annualized real GDP growth {cbcipi}, following {cipipr} '+
        f'in {prdt}. Over the past year, changes in private '+
        f'inventories {cipi_yr} from real GDP growth.')
write_txt(text_dir / 'cipi_contrib_text.txt', text)
print(text)

For example, from 1996 to 1999, gross domestic fixed investment added an average of 1.75 percentage points to annual real GDP growth. 

In the third quarter of 2023, gross domestic fixed investment contributed 0.40 percentage point to annualized real GDP growth, following a contribution of 1.35 points in the second quarter. Over the past year, gross fixed investment contributed 1.02 percentage point to real GDP growth.

In 2023 Q3, by type of gross fixed investment, private nonresidential did not contribute to annualized real GDP growth (see\cbox{yellow!70!orange!90!white}), private residential contributed 0.15 percentage point (see\cbox{blue!60!white}), and public contributed 0.25 percentage point (see\cbox{violet!60!black}). Over the past year, nonresidential or business gross fixed investment contributed 0.87 percentage point, residential subtracted 0.28 point, and public contributed 0.44 point. 

In 2023 Q3, changes in private inventories contributed 1.32 percentage points to annua

### Net Fixed Investment

In [27]:
d = {'W171RC': 'NetDomInv',  'W790RC': 'NetBusInv', 
     'W791RC': 'NetHHInv', 'A889RC': 'NetGovInv', 
     'A928RC': 'GrossInv_NIPA', 'W170RC': 'GrossInv', 
     'A262RC': 'CFC', 'W276RC': 'CFCBus', 'W279RC': 'CFCHH',
     'A264RC': 'CFCGov', 'W987RC': 'GrossBusInv', 
     'W988RC': 'GrossHHInv', 'A782RC': 'GrossGovInv'}
df = nipa_df(retrieve_table('T50100')['Data'], d.keys()).rename(d, axis=1)
df['GDP'] = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])
sh = df.divide(df['GDP'], axis=0) * 100
sh.loc['1989':].to_csv(data_dir / 'bea_nfi.csv', 
             index_label='date', float_format='%g')

### Imports and Exports

The 2023 comprehensive update split table 4.2.5. 

In [28]:
# Import and Export share of GDP
s = ['B020RC', 'B021RC', 'B648RC', 'LA000006']
s2 = ['A191RC']
df1 = nipa_df(retrieve_table('T40205A')['Data'], s)
df2 = nipa_df(retrieve_table('T40205B')['Data'], s)
df = pd.concat([df1, df2])
df['A191RC'] = nipa_df(retrieve_table('T10105')['Data'], s2)
df['EX'] = df['B020RC'] - df['LA000006']
df['IM'] = df['B021RC'] - df['B648RC']
data = (df.div(df['A191RC'], axis=0) * 100).loc['1989':]
data.to_csv(data_dir / 'eximgdp.csv', index_label='date')

In [29]:
data = pd.read_csv(data_dir / 'eximgdp.csv', index_col='date', 
                   parse_dates=True)
excol = 'green!60!teal!80!black'
imcol = 'blue!90!cyan'

node = end_node(data.IM, imcol, date='q', offset=0.35)
write_txt(text_dir / 'np_im_node.txt', node)
node = end_node(data.EX, excol)
write_txt(text_dir / 'np_ex_node.txt', node)

date = dtxt(data.index[-1])['qtr2']
pc_dt = '2019-10-01'
pc_date = dtxt(pd.to_datetime(pc_dt))['qtr1']
text = ('Nonpetroleum goods and services imports '+
        f'{c_line(imcol)} were equivalent to '+
        f'{data.IM.iloc[-1]:.1f} percent of GDP '+
        f'in {date}, while exports of nonpetroleum '+
        f'goods and services {c_line(excol)} were '+
        f'equivalent to {data.EX.iloc[-1]:.1f} percent '+
        f'of GDP. In {pc_date}, nonpetroleum imports '+
        f'were {data.IM.loc[pc_dt]:.1f} percent of GDP, '+
        f'and exports were {data.EX.loc[pc_dt]:.1f} percent.')
write_txt(text_dir / 'exim.txt', text)
print(text)

Nonpetroleum goods and services imports (see {\color{blue!90!cyan}\textbf{---}}) were equivalent to 12.9 percent of GDP in the third quarter of 2023, while exports of nonpetroleum goods and services (see {\color{green!60!teal!80!black}\textbf{---}}) were equivalent to 10.0 percent of GDP. In 2019 Q4, nonpetroleum imports were 13.0 percent of GDP, and exports were 10.6 percent.


### Goods Import Penetration

In [30]:
s = ['A353RC']
G = nipa_df(retrieve_table('T10205')['Data'], s).sort_index()

s = ['A253RC', 'A255RC', 'B647RC', 'LA000004', 'A650RC', 
     'B651RC', 'A652RC', 'A653RC', 'B648RC']
MX1 = nipa_df(retrieve_table('T40205A')['Data'], s).sort_index()
MX2 = nipa_df(retrieve_table('T40205B')['Data'], s).sort_index()
MX = pd.concat([MX1, MX2])

D = G['A353RC'] - MX['A253RC'] + MX['A255RC']
result = (MX['A255RC'] / D)

import_categories = ['B647RC', 'LA000004', 'A650RC', 'B651RC', 
                     'A652RC', 'A653RC', 'B648RC']
Msh = MX[import_categories].div(MX['A255RC'], axis=0)

Msh['Consumer'] = Msh['B647RC'] + Msh['A652RC'] + Msh['B651RC']
Msh['Capital'] = Msh['LA000004'] - Msh['B648RC'] + Msh['A650RC'] + Msh['A653RC']

final = Msh[['Consumer', 'Capital', 'B648RC']].multiply(result, axis=0) * 100

final.loc['1989':].to_csv(data_dir / 'goodsimpsh.csv', index_label='date')

In [79]:
final = pd.read_csv(data_dir / 'goodsimpsh.csv', index_col='date', 
                    parse_dates=True)
ltdate = final.index[-1]
ltdt = dtxt(ltdate)['qtr1']
cons, capi = [value_text(final[i].iloc[-1], style='eq') 
              for i in ['Consumer', 'Capital']]
oil = final['B648RC'].iloc[-1]
text = (f'In {ltdt}, the US imported nonpetroleum consumer goods '+
        f'{cons} of domestic consumption of '+
        'goods (see\cbox{cyan!40!white}). Petroleum-related imports claim '+
        f'{oil:.1f} percent (see\cbox{{purple}}), and imports of all other '+
        f'goods, primarily capital goods, industrial supplies, and materials, '+
        f'are {capi} (see\cbox{{blue!50!cyan}}).')
write_txt(text_dir / 'goodsimpsh1.txt', text)
print(text, '\n')

ch11 = (final.loc['2011-01-01'] - final.iloc[0])
cons11, pet11, oth11 = [value_text(ch11[i], adj='equivalent', threshold=0.1) 
                        for i in ['Consumer', 'B648RC', 'Capital']]
chlt = (final.iloc[-1] - final.loc['2011-01-01'])
conslt, petlt, othlt = [value_text(chlt[i], adj='equivalent', threshold=0.1) 
                        for i in ['Consumer', 'B648RC', 'Capital']]
text = ('From 1989 to 2011, imports of consumer goods excluding '+
        f'petroleum {cons11} of domestic consumption of goods, petroleum-'+
        f'related imports {pet11}, and all other goods imports {oth11}. \n\n'+
        f'Since 2011, imports of consumer goods {conslt} of domestic goods '+
        f'demand, imports of petroleum products {petlt}, and other '+
        f'imports {othlt}.')
write_txt(text_dir / 'goodsimpsh.txt', text)
print(text)

In 2023 Q3, the US imported nonpetroleum consumer goods equivalent to 14.8 percent of domestic consumption of goods (see\cbox{cyan!40!white}). Petroleum-related imports claim 2.7 percent (see\cbox{purple}), and imports of all other goods, primarily capital goods, industrial supplies, and materials, are equivalent to 14.3 percent (see\cbox{blue!50!cyan}). 

From 1989 to 2011, imports of consumer goods excluding petroleum increased by the equivalent of six percent of domestic consumption of goods, petroleum-related imports increased by the equivalent of 6.1 percent, and all other goods imports increased by the equivalent of 6.1 percent. 

Since 2011, imports of consumer goods decreased by the equivalent of 1.6 percent of domestic goods demand, imports of petroleum products decreased by the equivalent of 5.7 percent, and other imports decreased by the equivalent of 1.8 percent.


### International Trade Table

In [2]:
# Names of series
s = {'B020RC': 'Exports of Goods \& Services',
     'A253RC': 'Exports of Goods',
     'B638RC': '\hspace{2mm}Foods, Feeds, \& Beverages',
     'A639RC': '\hspace{2mm}Industrial Supplies \& Materials',
     'LA000006': '\hspace{4mm}Petroleum \& Products',
     'A640RC': '\hspace{2mm}Capital Goods, Except Automotive',
     'B641RC': '\hspace{2mm}Automotive Vehicles, \& Parts',
     'A642RC': '\hspace{2mm}Consumer Goods, Ex. Food \& Auto',
     'B690RC': '\hspace{4mm}Durable Goods',
     'B691RC': '\hspace{4mm}Nondurable Goods',
     'A646RC': 'Exports of Services',
     'Y800RC': '\hspace{2mm}Transport',
     'Y801RC': '\hspace{2mm}Travel',
     'B684RC': '\hspace{2mm}Intellectual Property Charges',
     'Y802RC': '\hspace{2mm}Other Business Services',
     'B021RC': 'Imports of Goods \& Services',
     'A255RC': 'Imports of Goods',
     'B647RC': '\hspace{2mm}Foods, Feeds, \& Beverages',
     'LA000004': '\hspace{2mm}Industrial Supplies \& Materials',
     'B648RC': '\hspace{4mm}Petroleum \& Products',
     'A650RC': '\hspace{2mm}Capital Goods, Except Automotive',
     'B651RC': '\hspace{2mm}Automotive Vehicles, \& Parts',
     'A652RC': '\hspace{2mm}Consumer Goods, Ex. Food \& Auto',
     'B697RC': '\hspace{4mm}Durable Goods',
     'B698RC': '\hspace{4mm}Nondurable Goods',
     'B656RC': 'Imports of Services',
     'Y804RC': '\hspace{2mm}Transport',
     'Y805RC': '\hspace{2mm}Travel',
     'B908RC': '\hspace{2mm}Intellectual Property Charges',
     'Y806RC': '\hspace{2mm}Other Business Services'}

# Collect data and divide by GDP
data = pd.concat([nipa_df(retrieve_table(i)['Data'], s) 
                  for i in ['T40205A', 'T40205B']])
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
df = data.div(gdp, axis=0) * 100
df.to_csv(data_dir/'exim.csv', index_label='date')

# Create table
tbl = pd.concat([df.iloc[i] for i in[-1, -2, -5]], axis=1)
tbl.columns = [dtxt(i)['qtr4'] for i in tbl.columns]
tbl['2016'] = df.loc['2016'].mean()
yrs = [(2012, 2013), (2005, 2006), (1998, 1999), (1989, 1993)]
for y1, y2 in yrs:
    tbl[f'{y1} --{str(y2)[2:]}'] = df.loc[str(y1):str(y2)].mean()
    
tbl = tbl.applymap('{:.2f}'.format)
tbl.index = tbl.index.map(s)
tbl.to_csv(data_dir / 'exim.tex', sep='&', 
           lineterminator='\\\ ', quotechar=' ')

In [31]:
#nipa_series_codes(retrieve_table('T10502'))

### GDP Contributions -- Long-term Average vs Recent Values bar chart

In [24]:
# expenditure approach data
ex_s = ['A191RX', 'DPCERX', 'A006RX', 'A019RX', 'A822RX']
ex_df = nipa_df(retrieve_table('T10106')['Data'], ex_s).loc['1989':]
ex_v = {'A191RX': ['\\textbf{Real GDP} \\\\[-1pt] \\textit{(annual rate)}', 
                   'red!95!black', 4], 
        'DPCERX': ['Consumer\\\\[-1pt] Spending', 'yellow!80!orange',  3],
        'A006RX': ['Private\\\\[-1pt] Investment', 'blue!70!black', 2],
        'A822RX': ['Government', 'cyan!50!white',  1],
        'A019RX': ['Net Exports', 'green!60!black', 0]}

# income approach data
inc_s = ['A261RC', 'A4002C', 'W271RC', 'indirect', 'A262RC']
s = ['A261RX']
rgdi = nipa_df(retrieve_table('T11706')['Data'], s).dropna()

s = ['A261RC', 'A4002C', 'W056RC', 'A107RC', 'W271RC', 'A262RC',
     'A4102C', 'A038RC']
df = nipa_df(retrieve_table('T11000')['Data'], s).dropna()

# Calculate indirect taxes net of transfers
df['indirect'] = df['W056RC'] - df['A107RC']

# Calculate GDI deflator from real GDI series
deflator = rgdi['A261RX'] / df['A261RC']
deflator = deflator / deflator.iloc[-1]
inc_df = df.multiply(deflator, axis=0)
inc_v = {'A261RC': ['\\textbf{Real GDI} \\\\ \\textit{(annual rate)}', 
                    'blue!95!black', 4], 
         'A4002C': ['Labor \\\\ Income', 'magenta!90!blue', 3],
         'W271RC': ['Profit', 'yellow!60!orange', 2],
         'indirect': ['Indirect Tax\\\[-1pt] -Subsidies', 'violet', 1],
         'A262RC': ['Depreciation', 'teal!60!white', 0]}

# houshold inputs data
hh_s = ['GDP_idx', 'POP_idx', 'EPOP_sa_idx', 'AAH_trend_idx', 'LPROD_idx']
hh_df = pd.read_csv(data_dir / 'gdpjobslvl.csv', index_col='date', 
                 parse_dates=True)[hh_s].dropna()
hh_v = {'GDP_idx': ['\\textbf{Real GDP}\\\\[-1pt] \\textit{(annual rate)}', 
                    'red!95!black', 4], 
        'POP_idx': ['Population', 'lime!90!green', 2],
        'EPOP_sa_idx': ['Employment\\\\[-1pt] Rate', 'green!30!teal!80!black', 1],
        'AAH_trend_idx': ['Average\\\\[-1pt] Workweek', 'blue', 0],
        'LPROD_idx': ['Labor\\\\[-1pt] Productivity', 'cyan!55!white', 3]}

# Charts for each approach
charts = {'exp': [ex_s, ex_df, ex_v], 'inc': [inc_s, inc_df, inc_v],
          'hhinp': [hh_s, hh_df, hh_v]}
bar_settings = 'xbar, bar width=3ex, bar shift=0pt, fill, '
c, ltc = {}, {}
for n, [s, df, v] in charts.items():
    df = df.dropna()
    if n != 'hhinp':
        contrib = (((((df.iloc[-1] - df.iloc[0])
                      / df[s[0]].iloc[0]) + 1)
                    ** (1/((len(df)-1)/4))) - 1) * 100
        c[n] = contrib
        ltch = growth_contrib(df, s[0]).iloc[-1]
        ltc[n] = ltch
    else:
        contrib = (((df.iloc[-1] / df.iloc[0]) 
                    ** (1/((len(df)) / 4))) - 1) * 100
        c[n] = contrib
        ltch = (((df.iloc[-1] / df.iloc[-2]) ** (4)) - 1) * 100
        ltch.name = df.index[-1]
        ltc[n] = ltch
    # Settings for plot
    cmax = max(contrib.max(), ltch.max())
    cmin = min(contrib.min(), ltch.min())
    crng = cmax - cmin
    cbuf = min(cmin, 0) #Buffer for text labels
    empty_neg = f'\\addplot[white!0] coordinates {{(-{cbuf:.2f}, 0)}};'
    txt = [empty_neg]
    ylx = (cmin - (crng*0.05)).round(3)
    for k, [name, color, y] in v.items():
        x = contrib[k].round(3)
        bar = f'\\addplot[{bar_settings} {color}] coordinates {{({x}, {y})}};'
        # Create ylabel and value label
        ylabel = ('\\node[left, align=right, text width=1.6cm] at '+
                  f'(axis cs:{ylx},{y}) {{{name}}};')
        # Node for latest quarter
        qval = ltch[k].round(3)
        qnode = ('\\addplot[only marks,mark=*, mark options={fill=white,draw=black}] '+
                 f'coordinates {{({qval},{y})}};')
        txt.append(bar)
        txt.append(ylabel)
        txt.append(qnode)
    nodes = '\n'.join(txt)
    write_txt(text_dir / f'gdp_{n}_chavg.txt', nodes)

    # Store latest date
    write_txt(text_dir / f'gdp_{n}_chavg_dt.txt', dtxt(df.index[-1])['qtr1'])

In [25]:
# Expenditure approach long-term change summary
exv = c['exp']
totch = value_text(exv['A191RX'], 'plain')
cs = value_text(exv['DPCERX'], 'contribution_to', ptype='pp')
gdpi = value_text(exv['A006RX'], 'contribution', ptype='pp', casual=True)
gov = value_text(exv['A822RX'], 'contribution', ptype='point', casual=True)
nx = value_text(exv['A019RX'], 'contribution', ptype='point')
text = (f'Since 1989, real GDP has grown {totch} per year. Over this '+
        f'period, consumer spending {cs} the annualized change, gross '+
        f'private domestic investment {gdpi}, government spending and '+
        f'investment {gov}, and net exports {nx}.')
write_txt(text_dir / 'gdp_contrib89_exp.txt', text)
print(text, '\n')

# Income approach text
inv = c['inc']
totch = value_text(inv['A261RC'], 'plain')
lab = value_text(inv['A4002C'], 'plain', ptype='pp')
prof = value_text(inv['W271RC'], 'plain', ptype='pp')
tax = value_text(inv['indirect'], 'plain', ptype='point')
cfc = value_text(inv['A262RC'], 'plain', ptype='point')
text = (f'Since 1989, real GDI has grown at an annualized rate of {totch}. '+
        f'Labor receives {lab} of this growth rate, profit '+
        f'claims {prof}, indirect taxes minus subsidies '+
        f'receive {tax}, and {cfc} go to depreciation. ')
write_txt(text_dir / 'gdp_contrib89_inc.txt', text)
print(text, '\n')


# Household inputs text
hhv = c['hhinp']
totch = value_text(hhv['GDP_idx'], 'plain')
popch = value_text(hhv['POP_idx'], 'plain')
lprodch = value_text(hhv['LPROD_idx'], 'plain')

text = (f'Real GDP growth of {totch} per year since 1989 is explained '+
        f'by annualized productivity growth of {lprodch} and population '+
        f'growth of {popch}. ')
write_txt(text_dir / 'gdp_contrib89_hh.txt', text)
print(text)

Since 1989, real GDP has grown 2.4 percent per year. Over this period, consumer spending contributed two percentage points to the annualized change, gross private domestic investment added 0.7 percentage point, government spending and investment added 0.4 point, and net exports subtracted 0.3 point. 

Since 1989, real GDI has grown at an annualized rate of 2.4 percent. Labor receives 1.5 percentage points of this growth rate, profit claims 0.8 percentage point, indirect taxes minus subsidies receive 0.2 point, and 0.6 point go to depreciation.  

Real GDP growth of 2.4 percent per year since 1989 is explained by annualized productivity growth of 1.5 percent and population growth of 0.9 percent. 


In [78]:
# Text HH inputs to latest published GDP growth number
df = ltc['hhinp']
ltdt = dtxt(df.name)['qtr1']
gdpch = value_text(df['GDP_idx'], 'plain')
epopnm = ('the higher employment rate' if df.EPOP_sa_idx > 0.4 
          else 'the lower employment rate' if df.EPOP_sa_idx < -0.4
          else 'the employment rate')
hoursnm = ('longer average workweeks' if df.AAH_trend_idx > 0.4 
          else 'the drop in hours worked' if df.AAH_trend_idx < -0.4
          else 'hours worked')
rn = {'EPOP_sa_idx': epopnm,
      'POP_idx': 'population growth',
      'AAH_trend_idx': hoursnm,
      'LPROD_idx': 'labor productivity'}
dfcats = df.drop('GDP_idx').rename(rn)
cat1 = dfcats.sort_values().index[-1]
cv1 = value_text(dfcats.sort_values().iloc[-1], 'contribution_to', 
                 ptype='pp', digits=2)
cat2 = dfcats.sort_values().index[-2].capitalize()
cv2 = value_text(dfcats.sort_values().iloc[-2], 'contribution', 
                 ptype='point', casual=True, digits=2)
cat3 = dfcats.sort_values().index[-3].capitalize()
cv3 = value_text(dfcats.sort_values().iloc[-3], 'contribution', 
                 ptype='point', casual=True, digits=2)
cat4 = dfcats.sort_values().index[-4].capitalize()
cv4 = value_text(dfcats.sort_values().iloc[-4], 'contribution', 
                 ptype='point', casual=True, digits=2)
small = dfcats[abs(dfcats) < 0.4].index # Small categories 
ext = '.'
if len(small) == 1:
    ext = f' and {cat3} {cv3}.'
if len(small) == 0:
    ext = f', {cat3} {cv3}, and {cat4} {cv4}.'
smalltxt = ''
if len(small) > 0:
    s1 = small[0].capitalize()
    if len(small) > 1:
        s2 = f' and {small[1]}'
        s1 = s1 + s2
    sverb = 'were' if len(small) > 1 else 'was'
    smalltxt = (f'{s1} {sverb} little changed in the period.')
text = (f'In {ltdt}, {cat1} {cv1} GDP growth of {gdpch}. {cat2} {cv2} '+
        f'in the period{ext} {smalltxt}')
write_txt(text_dir / 'gdp_contrib_lt_hh.txt', text)
print(text)

In 2023 Q3, labor productivity contributed 4.53 percentage points to GDP growth of 4.9 percent. Population growth added 0.58 point in the period. The employment rate and hours worked were little changed in the period.


### Contribution to change since 2019

In [549]:
# expenditure approach data
ex_s = ['A191RX', 'DPCERX', 'A006RX', 'A019RX', 'A822RX']
ex_df = nipa_df(retrieve_table('T10106')['Data'], ex_s).loc['1989':]
ex_v = {'A191RX': ['\\textbf{Real GDP} \\\\ \\textit{(annual rate)}', 
                   'red!95!black', 'white', 4.2], 
        'DPCERX': ['Consumer Spending', 'yellow!80!orange', 'black', 3],
        'A006RX': ['Private\\\\ Investment', 'blue!70!black', 'white', 2],
        'A822RX': ['Gov Spending \& Investment', 'cyan!50!white', 'black', 1],
        'A019RX': ['Net Exports', 'green!60!black', 'white', 0]}


# income approach data
inc_s = ['A261RC', 'A4002C', 'W271RC', 'indirect', 'A262RC']
s = ['A261RX']
rgdi = nipa_df(retrieve_table('T11706')['Data'], s).dropna()

s = ['A261RC', 'A4002C', 'W056RC', 'A107RC', 'W271RC', 'A262RC',
     'A4102C', 'A038RC']

df = nipa_df(retrieve_table('T11000')['Data'], s).dropna()

# Calculate indirect taxes net of transfers
df['indirect'] = df['W056RC'] - df['A107RC']

# Calculate GDI deflator from real GDI series
deflator = rgdi['A261RX'] / df['A261RC']
deflator = deflator / deflator.iloc[-1]
inc_df = df.multiply(deflator, axis=0)
inc_v = {'A261RC': ['\\textbf{Real GDI} \\\\ \\textit{(annual rate)}', 
                    'blue!95!black', 'white', 4.2], 
         'A4002C': ['Labor \\\\ Income', 'magenta!90!blue', 'white', 3],
         'W271RC': ['Profit', 'yellow!60!orange', 'black', 2],
         'indirect': ['Indirect Taxes Less Subsidies', 'violet', 'white', 1],
         'A262RC': ['Depreciation', 'teal!60!white', 'black', 0]}

# houshold inputs data
hh_s = ['GDP_idx', 'POP_idx', 'EPOP_sa_idx', 'AAH_trend_idx', 'LPROD_idx']
hh_df = pd.read_csv(data_dir / 'gdpjobslvl.csv', index_col='date', 
                 parse_dates=True)[hh_s].dropna()
hh_v = {'GDP_idx': ['\\textbf{Real GDP} \\\\ \\textit{(annual rate)}', 
                    'red!95!black', 'white', 4.2], 
        'POP_idx': ['Population', 'lime!90!green', 'black', 2],
        'EPOP_sa_idx': ['Employment Rate', 'green!30!teal!80!black', 'white', 1],
        'AAH_trend_idx': ['Average \\\\ Workweek', 'blue', 'white', 0],
        'LPROD_idx': ['Labor \\\\ Productivity', 'cyan!55!white', 'black', 3]}

# Charts for each approach
charts = {'exp': [ex_s, ex_df, ex_v], 'inc': [inc_s, inc_df, inc_v],
          'hhinp': [hh_s, hh_df, hh_v]}
for n, [s, df, v] in charts.items():
    d19 = df.loc['2019-10-01':]
    contrib = (((((d19.iloc[-1] - d19.iloc[0])
                  / d19[s[0]].iloc[0]) + 1)
                ** (1/((len(d19)-1)/4))) - 1) * 100
    # Settings for plot
    cmax = contrib.max()
    cmin = contrib.min()
    crng = cmax - cmin
    cbuf = max([(cmax - 0), (0 - cmin)]) * 0.68 #Buffer for text labels
    thresh = crng * 0.21 #Bigger bars labeled inside
    empty_neg = f'\\addplot[white!0] coordinates {{(-{cbuf:.2f}, 0)}};'
    empty_pos = f'\\addplot[white!0] coordinates {{({cbuf:.2f}, 0)}};'
    txt = [empty_neg, empty_pos]

    for k, [name, color, tcolor, y] in v.items():
        x = contrib[k].round(3)
        bar = f'\\addplot[{color}] coordinates {{({x}, {y})}};'
        vtc = 'black'
        tx = f'{contrib[k]:.1f}'
        if abs(x) > thresh:  # Some value labels inside of bars
            vt = f'\scriptsize \color{{{tcolor}}} \\textbf{{{tx}}}'
            inside = True
        else:
            vt = f'\scriptsize {tx}'
            inside = False
        if x > 0:
            ytlab = 'left, align=right'
            vtlab = 'left, align=right' if inside == True else 'right, align=right'
        else:
            ytlab = 'right, align=left'
            vtlab = 'right, align=left' if inside == True else 'left, align=left'
        # Create ylabel and value label
        ylabel = f'\\node[{ytlab}, text width=2.0cm] at (axis cs:0,{y}) {{{name}}};'
        vlabel = f'\\node[{vtlab}] at (axis cs:{x},{y}) {{{vt}}};'
        txt.append(bar)
        txt.append(ylabel)
        txt.append(vlabel)
    nodes = '\n'.join(txt)
    write_txt(text_dir / f'gdp_{n}_ch19.txt', nodes)

    # Store latest date
    write_txt(text_dir / f'gdp_{n}_ch19_dt.txt', dtxt(d19.index[-1])['qtr1'])

### Gross Domestic Income

In [607]:
# Annual contribution to growth
s = ['A261RX']
rgdi = nipa_df(retrieve_table('T11706A')['Data'], s).dropna()
s = ['A261RC', 'A4002C', 'W056RC', 'A107RC', 'W271RC', 'A262RC',
     'A4102C', 'A038RC']
df = nipa_df(retrieve_table('T11000A')['Data'], s).dropna()

# Calculate indirect taxes net of transfers
df['indirect'] = df['W056RC'] - df['A107RC']

# Calculate GDI deflator from real GDI series
deflator = rgdi['A261RX'] / df['A261RC']
deflator = deflator / deflator.iloc[-1]
df = df.multiply(deflator, axis=0)

# Calculate contributions to growth
contr = growth_contrib_ann(df, 'A261RC', freq='A')
contr.loc['1989':].to_csv(data_dir / 'gdi_comp_ann.csv', index_label='date')

# Text 
lt = contr.iloc[-1]
ltyr = contr.index[-1].year
gdi = value_text(lt['A261RC'])
lab = value_text(lt['A4002C'], 'contribution', ptype='pp')
prof = value_text(lt['W271RC'], 'contribution', ptype='pp', casual=True)
tax = value_text(lt['indirect'], 'contribution', ptype='point', casual=True)
cfc = value_text(lt['A262RC'], 'plain', ptype='point')
text = (f'In the latest full year of data, {ltyr}, real GDI {gdi}. '+
        f'Labor income {lab}, profit {prof}, '+
        f'indirect taxes less subsidies {tax}, and {cfc} '+
        f'went to depreciation. ')
write_txt(text_dir / 'gdi_ltyr_comp.txt', text)
print(text)

In the latest full year of data, 2022, real GDI increased 2.1 percent. Labor income contributed 0.1 percentage point, profit subtracted 0.2 percentage point, indirect taxes less subsidies added 1.6 points, and 0.6 point went to depreciation. 


In [582]:
# Collect data
s = ['A261RX']
rgdi = nipa_df(retrieve_table('T11706')['Data'], s).dropna()
s = ['A261RC', 'A4002C', 'W056RC', 'A107RC', 'W271RC', 'A262RC',
     'A4102C', 'A038RC']
df = nipa_df(retrieve_table('T11000')['Data'], s).dropna()
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']

# Calculate indirect taxes net of transfers
df['indirect'] = df['W056RC'] - df['A107RC']

# Calculate net domestic income (GDI less depreciation)
df['NDI'] = df['A261RC'] - df['A262RC']

# Calculate GDI deflator from real GDI series
deflator = rgdi['A261RX'] / df['A261RC']
deflator = deflator / deflator.iloc[-1]
df = df.multiply(deflator, axis=0)

# Calculate contributions to growth
contr = growth_contrib(df, 'A261RC')

# Save results
cols = ['A261RC', 'A4002C', 'A4102C', 'A038RC', 'W271RC',
        'indirect', 'W056RC', 'A107RC', 'A262RC', 'NDI']
contr.loc['1989':, cols].to_csv(data_dir / 'gdi.csv', index_label='date')
ctlt = contr[cols].iloc[-9:]
ctlt['label'] = [f'Q{i.quarter} \\\\ {i.year}' if (i.quarter == 1) | (i == ctlt.index[0]) 
                else dtxt(i)['qtr3'] for i in ctlt.index]
ctlt.to_csv(data_dir / 'gdi_latest.csv', index_label='date')

# Nodes for latest data
slist = ['A4002C', 'W271RC', 'A262RC', 'indirect']
nodes = selected_nodes(contr[slist].iloc[-1].to_frame().T)
write_txt(text_dir / 'gdi_latest_nodes.txt', nodes)

# Empty dots to set axis
f = contr[slist].iloc[-9:]
ymax = f[f>0].sum(axis=1).max()
ymin = f[f<0].sum(axis=1).min()
dt = dtxt(contr.index[-1])['datetime2']
dots = ('\\addplot[only marks, white, draw opacity=0, mark size=0.1pt]'+
        f' coordinates {{({{{dt}}},{ymax}) ({{{dt}}}, {ymin})}};')
write_txt(text_dir / 'gdi_latest_axis_dots.txt', dots)

# Calculate and store per capita data
dfpop = df[cols].div(pop, axis=0).dropna()
dfpop.to_csv(data_dir / 'gdi_pc_comp.csv', index_label='date')

# Store latest date
ltdt = dtxt(dfpop.index[-1])['qtr1']
prdt = dtxt(dfpop.index[-2])['qtr1']
write_txt(text_dir / 'gdi_ltdt.txt', ltdt)

In [558]:
# Per capita summary table
srs = {'A261RC': '\hspace{0.1mm} Gross Domestic Income', 
       'A4002C': '\hspace{-0.2mm} {\color{magenta!90!blue}\\textbf{---}} Labor Income', 
       'A4102C': '\hspace{4mm} Wages and Salaries',
       'A038RC': '\hspace{4mm} Supplements',
       'W271RC': '\hspace{-0.2mm} {\color{yellow!60!orange}\\textbf{---}} Profit', 
       'indirect': '\hspace{-0.1mm} {\color{violet}\\textbf{---}} Indirect Taxes', 
       'W056RC': '\hspace{4mm} Taxes on Production \& Imports',
       'A107RC': '\hspace{4mm} Less: Subsidies',
       'A262RC': '\hspace{-0.2mm} {\color{teal!60!white}\\textbf{---}} Depreciation'}
res = dfpop[srs.keys()].dropna() * 1000

dates = [res.index[-1], res.index[-5], '2019-10-01', '2012-01-01', 
         '2000-01-01', '1989-01-01']
tbl = (pd.concat([res.loc[dt] for dt in dates], axis=1)
         .rename(srs).applymap('{:,.0f}'.format))
tbl.columns = [dtxt(i)['qtr1'] for i in tbl.columns]
tbl.iloc[0, 0] = f'\${tbl.iloc[0, 0]}'
tbl.to_csv(data_dir / 'gdipc_levels.tex', sep='&', 
           lineterminator='\\\ ', quotechar=' ')

# GDI text
ltval = df['A261RC'].iloc[-1] / 1000
ltpc = dfpop['A261RC'].iloc[-1] * 1000
text = (f'of \${ltval:,.0f} billion in {ltdt}, which is '+
        f'\${ltpc:,.0f} per capita. ')
write_txt(text_dir / 'gdi_levels.txt', text)
print('\n', text, '\n')

# NDI text
ltval = df['NDI'].iloc[-1] / 1000
ltpc = dfpop['NDI'].iloc[-1] * 1000
text = (f'is \${ltval:,.0f} billion in {ltdt}, or '+
        f'\${ltpc:,.0f} per capita. ')
write_txt(text_dir / 'ndi_levels.txt', text)
print('\n', text, '\n')

# Labor and capital share
lsh = (df['A4002C'] / df['NDI']) * 100
csh = (df['W271RC'] / df['NDI']) * 100
dsh = (df['A262RC'] / df['A261RC']) * 100

# Text by income type
prdate = '2019-10-01'
l_pc = dfpop['A4002C'].iloc[-1] * 1000
l_pr = dfpop['A4002C'].loc[prdate] * 1000
k_pc = dfpop['W271RC'].iloc[-1] * 1000
k_pr = dfpop['W271RC'].loc[prdate] * 1000
t_pc = dfpop['indirect'].iloc[-1] * 1000
t_pr = dfpop['indirect'].loc[prdate] * 1000
d_pc = dfpop['A262RC'].iloc[-1] * 1000
d_pr = dfpop['A262RC'].loc[prdate] * 1000

text = (f'Labor receives {lsh.iloc[-1]:.1f} percent of NDI in {ltdt}. '+
        'Gross labor income per capita is '+
        f'\${l_pc:,.0f} in {ltdt} '+
        '(see {\color{magenta!90!blue}\\textbf{---}}) '+
        f'and \${l_pr:,.0f} in 2019 Q4, on an annualized, '+
        'seasonally-adjusted, and inflation-adjusted basis.\n\n '+
        f'Profits comprise {csh.iloc[-1]:.1f} percent of NDI in {ltdt}. '+
        f'Profits per person total \${k_pc:,.0f} in {ltdt} '+
        '(see {\color{yellow!60!orange}\\textbf{---}}) and '+
        f'\${k_pr:,.0f} in {prdt}, following the same adjustments. '+
        f'Indirect taxes less subsidies per capita total \${t_pc:,.0f} '+
        f'in {ltdt} (see {{\color{{violet}}\\textbf{{---}}}}) '+
        f'and \${t_pr:,.0f} in {prdt}.\n\nLastly, depreciation per '+
        f'capita is \${d_pc:,.0f} in {ltdt} (see '+
        '{\color{teal!60!white}\\textbf{---}}) and '+
        f'\${d_pr:,.0f} in {prdt}. Depreciation makes up '+
        f'{dsh.iloc[-1]:.1f} percent of GDI in {ltdt}. ')
write_txt(text_dir / 'gdi_levels_pc.txt', text)
print(text)


 of \$26,643 billion in 2023 Q2, which is \$79,528 per capita.  


 is \$22,086 billion in 2023 Q2, or \$65,926 per capita.  

Labor receives 64.2 percent of NDI in 2023 Q2. Gross labor income per capita is \$42,354 in 2023 Q2 (see {\color{magenta!90!blue}\textbf{---}}) and \$40,899 in 2019 Q4, on an annualized, seasonally-adjusted, and inflation-adjusted basis.

 Profits comprise 28.0 percent of NDI in 2023 Q2. Profits per person total \$18,433 in 2023 Q2 (see {\color{yellow!60!orange}\textbf{---}}) and \$18,175 in 2023 Q1, following the same adjustments. Indirect taxes less subsidies per capita total \$5,139 in 2023 Q2 (see {\color{violet}\textbf{---}}) and \$5,174 in 2023 Q1.

Lastly, depreciation per capita is \$13,602 in 2023 Q2 (see {\color{teal!60!white}\textbf{---}}) and \$12,418 in 2023 Q1. Depreciation makes up 17.1 percent of GDI in 2023 Q2. 


In [606]:
# GDI Growth Text
s = ['A261RC', 'A4002C', 'W271RC', 'indirect', 'A262RC']
df = contr[s]
gdi_lt = value_text(df["A261RC"].iloc[-1], adj='annual')
prval = prval_comp(df["A261RC"])

ltdt = dtxt(df.index[-1])['qtr2']
ltdt2 = dtxt(df.index[-1])['qtr1']
prdt = dtxt(df.index[-2])['qtr1']

text1 = (f'In {ltdt}, real GDI {gdi_lt}, following {prval}. ')  

l_lt = value_text(df["A4002C"].iloc[-1], style='contribution_to', 
                  digits=2, ptype='pp')
k_lt = value_text(df["W271RC"].iloc[-1], style='contribution', casual=True,
                  digits=2, ptype='pp')
t_lt = value_text(df['indirect'].iloc[-1], style='contribution', casual=True,
                  digits=2, ptype='point')
d_lt = value_text(df['A262RC'].iloc[-1], style='plain', 
                  digits=2, ptype='point')
    
text = (f'{text1}In the latest quarter, labor income {l_lt} annualized growth, '+
        f'profit {k_lt}, changes in indirect tax revenue '+
        f'and subsidies {t_lt}, and {d_lt} went to depreciation growth. ')
write_txt(text_dir / 'gdi_growth_comp.txt', text)
print(text) 

In the first quarter of 2022, real GDI increased at an annual rate of 2.1 percent, following an increase of 6.1 percent in 2021 Q1, and a decrease of 2.3 percent in 2020 Q1. In the latest quarter, labor income contributed 0.06 percentage point to annualized growth, profit subtracted 0.17 percentage point, changes in indirect tax revenue and subsidies added 1.62 points, and 0.61 point went to depreciation growth. 


### Consumer Spending Overview (Levels)

In [65]:
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']

n = {'DPCERC': 'Total',
     'DGDSRC': 'Goods',
     'DSERRC': 'Services',
     'DHUTRC': 'Housing',
     'A011RC': 'ResInv',
     'DMOTRC': 'MotorVeh',
     'DFDHRC': 'Furn',
     'DREQRC': 'RecDG',
     'DFXARC': 'Groc',
     'DCLORC': 'Cloth',
     'DHLCRC': 'Health',
     'DTRSRC': 'Transp',
     'DRCARC': 'RecSer',
     'DFSARC': 'FoodAcc',
     'DIFSRC': 'FinIns'}
n2 = {k[:-2] + 'RA': v for k, v in n.items()}
s = list(n.keys())
s2 = list(n2.keys())

othserv = lambda x: x['Services'] - x['Housing']
shelter = lambda x: x['Housing'] + x['ResInv']

df = (nipa_df(retrieve_table('T10505')['Data'], s)
      .rename(n, axis=1))
df2 = (nipa_df(retrieve_table('T10503')['Data'], s2)
       .rename(n2, axis=1))
real = ((df2 / df2.iloc[-1]) * df.iloc[-1]).assign(OTHSERV = othserv, SHELTER = shelter)
pp = real.divide(pop, axis=0)

keep_col = ['Goods', 'Services', 'Housing', 'ResInv', 'OTHSERV', 'SHELTER']
pp.loc['1989':, keep_col].to_csv(data_dir / 'pce_levels.csv', index_label='date')

lttot = real.Total.iloc[-1] / 1_000_000
prtot = real.Total.iloc[-2] / 1_000_000
pctot = real.Total.loc['2019-10-01'] / 1_000_000
ltdate = dtxt(real.index[-1])['qtr1']
prdate = dtxt(real.index[-2])['qtr1']
totpp = pp.Total.iloc[-1] * 1_000
goodpp = pp.Goods.iloc[-1] * 1_000
servpp = pp.Services.iloc[-1] * 1_000
goodpppc = pp.Goods.loc['2019-10-01'] * 1_000
servpppc = pp.Services.loc['2019-10-01'] * 1_000

text = ('Total consumer spending is '+
        f'\${lttot:.1f} trillion in {ltdate}, compared to a price-adjusted '+
        f'\${prtot:.1f} trillion in {prdate} and \${pctot:.1f} trillion in 2019 Q4. '+
        'On a per person basis, consumer spending is '+
        f'\${totpp:,.0f} in {ltdate}, of which \${goodpp:,.0f} are spent on goods '+
        '(see {\color{red}\\textbf{---}}) and '+
        f'\${servpp:,.0f} on services '+
        '(see {\color{orange}\\textbf{---}}). In the fourth quarter of 2019, '+
        f'before the pandemic, consumer spending on goods was \${goodpppc:,.0f} '+
        f'per person, and spending on services was \${servpppc:,.0f} per person, '+
        'after adjusting for inflation. ')
write_txt(text_dir / 'pce_levels.txt', text)
print(text)

hult = pp['Housing'].iloc[-1] * 1_000
hupc = pp['Housing'].loc['2019-10-01'] * 1_000
rfilt = pp['ResInv'].iloc[-1] * 1_000
rfipc = pp['ResInv'].loc['2019-10-01'] * 1_000

text = ('Within consumer spending on services, housing and utilities spending '+
        f'totals \${hult:,.0f} on an annualized and per person basis in {ltdate} '+
        '(see {\color{green!60!blue}\\textbf{---}}) '+
        f'and \${hupc:,.0f} in 2019 Q4. Construction or improvement '+
        'of housing is considered residential fixed investment, not '+
        'consumer spending, but can be combined with spending to analyze '+
        'patterns in shelter costs. In '+
        f'{ltdate}, residential investment totals \${rfilt:,.0f} per person '+
        '(see {\color{blue!80!black}\\textbf{---}}), '+
        f'compared to \${rfipc:,.0f} in the pre-COVID data covering 2019 Q4. ')
write_txt(text_dir / 'pce2_levels.txt', text)
print('\n', text)

othlt = pp['OTHSERV'].iloc[-1] * 1_000
othpr = pp['OTHSERV'].iloc[-2] * 1_000
othpc = pp.loc['2019-10-01', 'OTHSERV'] * 1_000
chval = ((othlt / othpc) - 1) * 100
chtxt = value_text(chval)
shellt = pp['SHELTER'].iloc[-1] * 1_000
shelpr = pp['SHELTER'].iloc[-2] * 1_000
shelpc = pp.loc['2019-10-01', 'SHELTER'] * 1_000
shelmax = pp['SHELTER'].max() * 1_000
shelmaxdt = dtxt(pp.SHELTER.idxmax())['qtr2']

text = ('Consumer spending on services other than housing and utilities totals \$'+
        f'{othlt:,.0f} per person, on an annualized basis, in {ltdate} '+
        '(see {\color{blue!75!white}\\textbf{---}}), compared to an '+
        f'inflation-adjusted \${othpr:,.0f} in {prdate}, and \${othpc:,.0f} in '+
        f'2019 Q4. Spending on non-housing services has {chtxt} since 2019 Q4. '+
        '\n\nShelter costs, which combine housing, utilities, and residential fixed '+
        f'investment, are \${shellt:,.0f} per person in {ltdate} '+
        '(see {\color{green!85!blue}\\textbf{---}})'+
        f', \${shelpr:,.0f} '+
        f'in {prdate}, and \${shelpc:,.0f} in 2019 Q4. Shelter spending peaked at '+
        f'\${shelmax:,.0f} per person in {shelmaxdt}, during the housing bubble.')
write_txt(text_dir / 'pce3_levels.txt', text)
print('\n', text)

Total consumer spending is \$18.7 trillion in 2023 Q3, compared to a price-adjusted \$18.6 trillion in 2023 Q2 and \$17.0 trillion in 2019 Q4. On a per person basis, consumer spending is \$55,840 in 2023 Q3, of which \$18,569 are spent on goods (see {\color{red}\textbf{---}}) and \$37,270 on services (see {\color{orange}\textbf{---}}). In the fourth quarter of 2019, before the pandemic, consumer spending on goods was \$15,909 per person, and spending on services was \$35,542 per person, after adjusting for inflation. 

 Within consumer spending on services, housing and utilities spending totals \$9,877 on an annualized and per person basis in 2023 Q3 (see {\color{green!60!blue}\textbf{---}}) and \$9,405 in 2019 Q4. Construction or improvement of housing is considered residential fixed investment, not consumer spending, but can be combined with spending to analyze patterns in shelter costs. In 2023 Q3, residential investment totals \$3,208 per person (see {\color{blue!80!black}\textbf{-

In [35]:
n = {'Total': 'Consumer Spending',
     'Goods': '\hspace*{-0.6mm} {\color{red}\\textbf{---}} Goods',
     'MotorVeh': '\hspace{4mm} Motor Vehicles \& Parts',
     'Furn': '\hspace{4mm} Furniture \& HH Equipment',
     'RecDG': '\hspace{4mm} Recreational Durable Goods',
     'Groc': '\hspace{4mm} Groceries',
     'Cloth': '\hspace{4mm} Clothes \& Shoes',
     'OTHSERV': '\hspace*{-0.6mm} {\color{blue!75!white}\\textbf{---}} Services Excluding Shelter',
     'Health': '\hspace{4mm} Health Care Services',
     'Transp': '\hspace{4mm} Transportation',
     'RecSer': '\hspace{4mm} Recreational',
     'FoodAcc': '\hspace{4mm} Food \& Accommodations',
     'FinIns': '\hspace{4mm} Financial \& Insurance',
     'SHELTER': '\hspace*{-0.6mm} {\color{green!85!blue}\\textbf{---}} Shelter Including Investment',
     'Housing': '\hspace{4mm} Housing Services \& Utilities ',
     'ResInv': '\hspace{4mm} Residential Fixed Investment'}

res = pp[n.keys()].rename(n, axis=1) * 1_000
lt = res.iloc[-1]
lt.name = dtxt(lt.name)['qtr1']
pr = res.iloc[-2]
pr.name = dtxt(pr.name)['qtr1']
py = res.iloc[-5]
py.name = dtxt(py.name)['qtr1']
pc = res.loc['2019-10-01']
pc.name = dtxt(pc.name)['qtr1']
p00 = res.loc['2000-01-01']
p00.name = dtxt(p00.name)['qtr1']
fi = res.loc['1989-01-01']
fi.name = dtxt(fi.name)['qtr1']
table = pd.concat([lt, pr, py, pc, p00, fi], axis=1).applymap('{:,.0f}'.format)
table.columns = table.columns.str.replace(' ', '\n')
table.iloc[0, 0] = f'\${table.iloc[0, 0]}'
table.to_csv(data_dir / 'pce_levels.tex', sep='&', lineterminator='\\\ ', quotechar=' ')
#table

### Consumer Spending and Residential Fixed Investment

In [36]:
s = ['DNPIRY', 'DSERRY', 'DPCERY', 'DGDSRY', 'DMOTRY',
     'DFDHRY', 'DREQRY', 'DFXARY', 'DCLORY', 'DHLCRY',
     'DTRSRY', 'DRCARY', 'DFSARY', 'DIFSRY', 'DHUTRY',
     'A011RY']

n = {'TOTAL': '& Consumer Spending',
     'DGDSRY': '\cbox{red} & Goods',
     'DMOTRY': '& \hspace{1mm} Motor Vehicles \& Parts',
     'DFDHRY': '& \hspace{1mm} Furniture \& HH Equipment',
     'DREQRY': '& \hspace{1mm} Recreational Durable Goods',
     'DFXARY': '& \hspace{1mm} Groceries',
     'DCLORY': '& \hspace{1mm} Clothes \& Shoes',
     'OTHSERV': '\cbox{blue!75!white} & Services Excluding Shelter',
     'DHLCRY': '& \hspace{1mm} Health Care Services',
     'DTRSRY': '& \hspace{1mm} Transportation',
     'DRCARY': '& \hspace{1mm} Recreational',
     'DFSARY': '& \hspace{1mm} Food \& Accommodations',
     'DIFSRY': '& \hspace{1mm} Financial \& Insurance',
     'SHELTER': '\cbox{green!85!blue} & Shelter Including Investment',
     'DHUTRY': '& \hspace{1mm} Housing Services \& Utilities ',
     'A011RY': '& \hspace{1mm} Residential Fixed Investment'}

total = lambda x: x['DPCERY']
othserv = lambda x: x['DSERRY'] - x['DHUTRY']
shelter = lambda x: x['DHUTRY'] + x['A011RY']

df = (nipa_df(retrieve_table('T10502')['Data'], s)
      .assign(TOTAL = total, OTHSERV = othserv, SHELTER = shelter)
      [list(n.keys())])

(df.loc['1988':, ['DGDSRY', 'OTHSERV', 'SHELTER']]
 .rolling(4).mean().dropna()
 .to_csv(data_dir / 'pce_ma.csv', index_label='date', float_format='%g'))

data = pd.concat([df.iloc[-3:].iloc[::-1].T, df.iloc[-5]], axis=1)
cols = [f'& {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]
data.columns = cols
data['1-year'] = df.rolling(4).mean().iloc[-1]
data['10-year'] = df.rolling(40).mean().iloc[-1]
data['30-year'] = df.rolling(120).mean().iloc[-1]
data.index = data.index.map(n)
(data.round(2).applymap('{:,.2f}'.format)
 .to_csv(data_dir / 'pce.tex', sep='&', lineterminator='\\\ ', quotechar=' '))

ld = dtxt(df.index[-1])['qtr1']
prd = dtxt(df.index[-2])['qtr1']
ld2 = dtxt(df.index[-1])['qtr2']

totlt = df['TOTAL'].iloc[-1]
totltt = value_text(totlt, style='contribution_to', ptype='pp', 
                    digits=1, threshold=0.1)
totpr = df['TOTAL'].iloc[-2]
totprt = value_text(totpr, style='contribution', ptype='pp', 
                    digits=1, threshold=0.1)
totpc = df.loc['2019-10-01', 'TOTAL']
totpct = value_text(totpc, style='contribution_of', ptype='pp', 
                    digits=1, casual=True, threshold=0.1)

txt1 = (f'These categories {totltt} GDP growth in {ld} and {totprt} in '+
        f'{prd}, compared to {totpct} in 2019 Q4, before the pandemic.')
write_txt(text_dir / 'pce1.txt', txt1)
print(txt1)

gdslt = df['DGDSRY'].iloc[-1]
gdsltt = value_text(gdslt, style='contribution_to', ptype='pp', 
                    digits=1, threshold=0.1)
serlt = df['OTHSERV'].iloc[-1]
serltt = value_text(serlt, style='contribution', ptype='pp', digits=1, 
                    casual=True, threshold=0.1)
shelt = df['SHELTER'].iloc[-1]
sheltt = value_text(shelt, style='contribution', ptype='pp', 
                    digits=1, threshold=0.1)

txt2 = (f'In {ld2}, household spending on goods {gdsltt} GDP growth, '+
         'household spending on services other than housing and utilities '+
        f'{serltt}, and shelter spending and investment {sheltt}.')
write_txt(text_dir / 'pce2.txt', txt2)
print('\n', txt2)

These categories contributed 2.7 percentage points to GDP growth in 2023 Q3 and contributed 0.6 percentage point in 2023 Q2, compared to an addition of 1.7 percentage points in 2019 Q4, before the pandemic.

 In the third quarter of 2023, household spending on goods contributed 1.1 percentage points to GDP growth, household spending on services other than housing and utilities added 1.2 percentage points, and shelter spending and investment contributed 0.6 percentage point.


### Trade Contribution to GDP

In [37]:
s = ['A019RY', 'A253RY', 'A646RY', 'A255RY', 'A656RY']
df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'nx.csv', index_label='date')

sl = [('A253RY', 'contribution_to'), ('A646RY', 'contribution'), 
      ('A255RY', 'contribution_to'), ('A656RY', 'contribution')]

d = {}
for s, style in sl:
    d[s] = value_text(df[s].iloc[-1], style, 'pp', digits=2)
    
ldate = dtxt(df.index[-1])['qtr2']

text = (f"Goods exports {d['A253RY']} GDP growth in {ldate} while "+
        f"services exports {d['A646RY']}. Good imports {d['A255RY']} "+
        f"GDP growth and services imports {d['A656RY']}.")
write_txt(text_dir / 'trade.txt', text)
print(text)

Goods exports contributed 0.54 percentage point to GDP growth in the third quarter of 2023 while services exports contributed 0.14 percentage point. Good imports subtracted 0.63 percentage point from GDP growth and services imports subtracted 0.12 percentage point.


### Business investment

In [19]:
# Contributions to growth
s = ['Y001RY', 'A009RY', 'Y033RY']
df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'businv.csv', index_label='date')

# By category/type
s1 = ['Y001RC', 'B009RC', 'Y033RC']
s = s1 + ['A008RC']
data = nipa_df(retrieve_table('T10105')['Data'], s)
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
sh = data.divide(gdp, axis=0) * 100
sh.loc['1989':].to_csv(data_dir / 'businvsh.csv', index_label='date')

# End nodes
colors = {'Y001RC': 'violet', 'B009RC': 'yellow!50!orange',
          'Y033RC': 'cyan!60!white'}
adj = node_adj(sh[s])
date = {n: None for n, c in colors.items()}
imax = sh[s1].iloc[-1].idxmax()
adj[imax] += 0.32
date[imax] = 'q'
nodes = '\n'.join([end_node(sh[n], c, date=date[n], offset=adj[n])
                   for n, c in colors.items()])
write_txt(text_dir / f'bus_inv_sh_nodes.txt', nodes)

# Text
cl = {n: c_line(c) for n, c in colors.items()}
ltdt = dtxt(sh.index[-1])['qtr1']
levels, shgdp = {}, {}
for series in s1:
    level_val = data[series].iloc[-1] / 1_000
    levels[series] = f"\${level_val:,.0f} billion"
    shgdp[series] = f"{value_text(sh[series].iloc[-1], 'plain')} of GDP"
text = ('Business fixed investment encompasses '+
        'structures, equipment, and intellectual '+
        'property, such as software and R\&D. Annualized '+
        f'investment in structures is {levels["B009RC"]} in {ltdt}, '+
        f'representing {shgdp["B009RC"]} {cl["B009RC"]}. Equipment '+
        f'investment is {levels["Y033RC"]} or {shgdp["Y033RC"]} '+
        f'{cl["Y033RC"]}, and intellectual property investment '+
        f'is {levels["Y001RC"]}, which is {shgdp["Y001RC"]} {cl["Y001RC"]}.')
write_txt(text_dir / 'businv_sh.txt', text)
print(text)

Business fixed investment encompasses structures, equipment, and intellectual property, such as software and R\&D. Annualized investment in structures is \$832 billion in 2023 Q3, representing three percent of GDP (see {\color{yellow!50!orange}\textbf{---}}). Equipment investment is \$1,384 billion or five percent of GDP (see {\color{cyan!60!white}\textbf{---}}), and intellectual property investment is \$1,502 billion, which is 5.4 percent of GDP (see {\color{violet}\textbf{---}}).


In [16]:
# Gross vs Net
s = ['W790RC', 'W276RC', 'W987RC']
df = nipa_df(retrieve_table('T50100')['Data'], s).join(data['A008RC'])
df['NFI'] = (df['A008RC'] - df['W276RC']) # Net Fixed Investment
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
res = (df.div(gdp, axis=0) * 100).dropna()
res.loc['1989':].to_csv(data_dir / 'businv_main.csv', index_label='date')

ltdt1 = dtxt(df.index[-1])['qtr1']
ltdt2 = dtxt(df.index[-1])['qtr2']
shgdp, levels = {}, {}
for series in ['W790RC', 'W276RC', 'W987RC', 'A008RC', 'NFI']:
    level_val = df[series].iloc[-1] / 1_000
    n = ''
    if level_val < 0:
        n = 'negative '
        level_val = abs(level_val)
    levels[series] = f"{n}\${level_val:,.0f} billion"
    shgdp[series] = f"{res[series].iloc[-1]:.1f} percent of GDP"
    
diff = cagr(df.loc['2019-10-01':, 'W987RC'])
gdt = value_text(diff, adj='annual')
diff2 = cagr(df.loc['2019-10-01':, 'NFI'])
if diff2 < -95:
    ndt = 'but collapsed completely'
else:
    ndt = 'and ' + value_text(diff2, adj='annual')
gpcl = f"{n}\${df.loc['2019-10-01', 'W987RC'] / 1_000:,.0f} billion"
npcl = f"{n}\${df.loc['2019-10-01', 'NFI'] / 1_000:,.0f} billion"
if diff < 0:
    txt = 'as gross investment outpaced depreciation'
else:
    txt = 'as growth of depreciation costs outpaced the increase in gross investment'
    
text = (f'In {ltdt2}, gross private business investment totals '+
        f'{levels["W987RC"]} on a seasonally-adjusted annualized basis, '+
        f'which is {shgdp["W987RC"]} '+
        '(see {\color{blue!60!violet}\\textbf{---}}). Private business investment '+
        f'in fixed assets totals {levels["A008RC"]}, or {shgdp["A008RC"]} '+
        '(see {\color{cyan!80!white}\\textbf{---}}). Private business depreciation '+
        f'totals {levels["W276RC"]} in the quarter, or {shgdp["W276RC"]} '+
        '(see {\color{magenta}\\textbf{---}}). '+
        f'As a result, net fixed investment is {levels["NFI"]}, or {shgdp["NFI"]} '+
        '(see {\color{green!80!blue}\\textbf{---}}).\n\n '+
        'In 2019 Q4, prior to the COVID-19 pandemic, private business gross '+
        f'investment was {gpcl}. Since 2019 Q4, gross investment {gdt}. '+
        f'Net fixed investment was {npcl} in 2019 Q4, {ndt} from 2019 Q4 to {ltdt1}, '+
        f'{txt}. ')
write_txt(text_dir / 'businv_levels.txt', text)
print(text)  

In the third quarter of 2023, gross private business investment totals \$3,709 billion on a seasonally-adjusted annualized basis, which is 13.4 percent of GDP (see {\color{blue!60!violet}\textbf{---}}). Private business investment in fixed assets totals \$3,718 billion, or 13.5 percent of GDP (see {\color{cyan!80!white}\textbf{---}}). Private business depreciation totals \$2,992 billion in the quarter, or 10.8 percent of GDP (see {\color{magenta}\textbf{---}}). As a result, net fixed investment is \$726 billion, or 2.6 percent of GDP (see {\color{green!80!blue}\textbf{---}}).

 In 2019 Q4, prior to the COVID-19 pandemic, private business gross investment was \$2,935 billion. Since 2019 Q4, gross investment increased at an annual rate of 6.4 percent. Net fixed investment was \$711 billion in 2019 Q4, and increased at an annual rate of 0.6 percent from 2019 Q4 to 2023 Q3, as growth of depreciation costs outpaced the increase in gross investment. 


### Business Investment Table and Text

In [22]:
# Contribution to growth by business investment type
s = ['A008RY', 'A009RY', 'Y033RY', 'Y034RY', 'B935RY', 
     'A680RY', 'A681RY', 'Y001RY', 'B985RY', 'Y006RY']

n = { 'A008RY': 'Total',
      'A009RY': '\hspace{-2mm}\cbox{yellow!50!orange}Structures',
      'Y033RY': '\hspace{-2mm}\cbox{cyan!60!white}Equipment',
      'Y034RY': '\hspace{4mm} Information Processing',
      'B935RY': '\hspace{6mm} Computers \& Peripherals',
      'A680RY': '\hspace{4mm} Industrial Equipment',
      'A681RY': '\hspace{4mm} Transportation Equipment',
      'Y001RY': '\hspace{-2mm}\cbox{violet}Intellectual Property Products',
      'B985RY': '\hspace{4mm} Software',
      'Y006RY': '\hspace{4mm} Research \& Development'}

df = (nipa_df(retrieve_table('T10502')['Data'], s)
      [list(n.keys())])
data = df.iloc[-3:].iloc[::-1].T
data[df.index[-5]] = df.iloc[-5]
data[df.index[-9]] = df.iloc[-9]
data.columns = [f' {q.year} Q{q.quarter}' 
                if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
                for i, q in enumerate(data.columns)]
data['1-year'] = df.rolling(4).mean().iloc[-1]
data['10-year'] = df.rolling(40).mean().iloc[-1]
data['30-year'] = df.rolling(120).mean().iloc[-1]
data.index = data.index.map(n)
res = data.applymap('{:,.2f}'.format)
res.to_csv(data_dir / 'businv.tex', sep='&', 
           lineterminator='\\\ ', quotechar=' ')

In [52]:
# Text
thresh = 0.02
sl = [('A008RY', 'contribution_to'), ('A009RY', 'contribution_to'), 
      ('Y033RY', 'contribution'), ('Y001RY', 'contribution'), 
      ('A680RY', 'contribution')]
d = {}
for s, style in sl:
    value = df[s].iloc[-1]
    d[s] = (value_text(df[s].iloc[-1], style,
           ptype='pp', digits=2, threshold=thresh))

t_1y = value_text(df['A008RY'].iloc[-4:].mean(), 'contribution_of', adj='average',
           ptype='pp', digits=2, threshold=thresh).replace('an', 'the')
ld = dtxt(df.index[-1])['qtr1']
ld2 = dtxt(df.index[-1])['qtr2']
val1y = df.A008RY.rolling(4).mean().iloc[-1]
vallt = df.A008RY.iloc[-1]
compare = compare_text(vallt, val1y, [0.1, 0.5, 2.0])
colors = {'Y001RY': 'violet', 'A009RY': 'yellow!50!orange',
          'Y033RY': 'cyan!60!white'}
cb = {n: c_box(c) for n, c in colors.items()}
d['Y001RY'] = d['Y001RY'].replace('contributed', 'added')
annu = 'annualized ' if abs(df.A008RY.iloc[-1]) > thresh else ''
v9200 = value_text(df.loc['1992':'2000', 'A008RY'].mean(), 'plain', ptype='pp')
v30 = value_text(data.loc['Total', '30-year'], 'contribution', ptype='pp')
v19 = value_text(df.loc['2019':, 'A008RY'].mean(), 'contribution', threshold=0.02, 
                 casual=True, ptype='pp').replace('ed ', 'ed an average of ')
v1 = value_text(data.loc['Total', '1-year'], 'contribution', ptype='pp', 
                threshold=0.02)
text = ('From 1992 to 2000, business fixed investment '+
        f'contributed an average of {v9200} to GDP growth. '+
        'Over the past three decades, the category '+
        f'{v30}, on average. Business investment {v19} since 2019, '+
        f'and {v1} over the past year.\n\n'+
        f"Private business gross fixed investment {d['A008RY']} "+
        f"{annu}GDP growth in {ld}. Within the category, investment "+
        f"in structures {d['A009RY']} {cb['A009RY']}, investment in "+
        f"equipment {d['Y033RY']} {cb['Y033RY']}, and investment "+
        f"in intellectual property {d['Y001RY']} {cb['Y001RY']}.")
write_txt(text_dir / 'businv.txt', text)
print(text)

From 1992 to 2000, business fixed investment contributed an average of 1.1 percentage points to GDP growth. Over the past three decades, the category contributed 0.6 percentage point, on average. Business investment added an average of 0.4 percentage point since 2019, and contributed 0.5 percentage point over the past year.

Private business gross fixed investment did not contribute to GDP growth in 2023 Q3. Within the category, investment in structures contributed 0.05 percentage point to (see\cbox{yellow!50!orange}), investment in equipment subtracted 0.19 percentage point (see\cbox{cyan!60!white}), and investment in intellectual property added 0.14 percentage point (see\cbox{violet}).


### Government spending and investment

In [44]:
n = {'A822RY': 'Consolidated Government Total',
     'A823RY': '\hspace{2mm}Federal Total',
     'A824RY': '\cbox{blue!60!black}National Defense',
     'A997RY': '\hspace{6mm}Consumption Expenditures',
     'A788RY': '\hspace{6mm}Gross Investment',
     'A825RY': '\cbox{green!85!black}Federal Non-Defense',
     'A542RY': '\hspace{6mm}Consumption Expenditures',
     'A798RY': '\hspace{6mm}Gross Investment',
     'A829RY': '\hspace{-2mm}\cbox{purple!70!magenta}State \& Local Total',
     'A991RY': '\hspace{4mm}Consumption Expenditures',
     'A799RY': '\hspace{4mm}Gross Investment'}

df = nipa_df(retrieve_table('T10502')['Data'], 
             list(n.keys()) + ['A191RL'])
df.rolling(4).mean().loc['1989':].to_csv(data_dir / 'gov.csv', 
                                         index_label='date')

In [45]:
ltdt = dtxt(df.index[-1])['qtr2']
ltdt2 = dtxt(df.index[-1])['qtr1']
tot = df.A822RY
gdp = df['A191RL'].iloc[-1]
oneyr = value_text(tot.iloc[-4:].mean(), 'contribution', 'pp', 
                   threshold=0.01, digits=2)
v89 = tot.loc['1989':].mean()
totval = value_text(tot.iloc[-1], 'contribution_to', 'pp', 
                    threshold=0.01, digits=2)
defval = value_text(df['A824RY'].iloc[-4:].mean(), 'contribution', 'pp', 
                   threshold=0.01, digits=2)
defcb = c_box('blue!60!black')
fedval = value_text(df['A825RY'].iloc[-4:].mean(), 'contribution', 'pp', 
                   threshold=0.01, digits=2)
fedcb = c_box('green!85!black')
slgval = value_text(df['A829RY'].iloc[-4:].mean(), 'contribution', 'pp', 
                   threshold=0.01, digits=2)
slgcb = c_box('purple!70!magenta')
text = ('Government consumption and investment directly affect economic '+
        f'growth in the short-term. In {ltdt}, government consumption spending '+
        f'and investment {totval} the real GDP growth rate of {gdp:.1f} '+
        'percent. Over the last four quarters, government consumption '+
        f'and investment {oneyr} to economic growth, on average. '+
        f'Since 1989, the average contribution has been {v89:.2f} '+
        f'percentage points.\n\nOver the four quarters ending {ltdt2}, '+
        f'by level of government, national defense {defval} {defcb}, '+
        f'federal non-defense {fedval} {fedcb}, and state and '+
        f'local government {slgval} {slgcb}.')
write_txt(text_dir / 'gov.txt', text)
print(text)

# Table 
res = df[n.keys()]
data = res.iloc[-3:].iloc[::-1].T
data[res.index[-5]] = res.iloc[-5]
data[res.index[-9]] = res.iloc[-9]
cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]

data.columns = cols
data['1-year'] = res.rolling(4).mean().iloc[-1].round(2)
data['10-year'] = res.rolling(40).mean().iloc[-1].round(2)
data['30-year'] = res.rolling(120).mean().iloc[-1].round(2)
data.index = data.index.map(n)
data = data.applymap('{:.2f}'.format)
data.to_csv(data_dir / 'gov.tex', sep='&', lineterminator='\\\ ', quotechar=' ')

Government consumption and investment directly affect economic growth in the short-term. In the third quarter of 2023, government consumption spending and investment contributed 0.79 percentage point to the real GDP growth rate of 4.9 percent. Over the last four quarters, government consumption and investment contributed 0.77 percentage point to economic growth, on average. Since 1989, the average contribution has been 0.25 percentage points.

Over the four quarters ending 2023 Q3, by level of government, national defense contributed 0.18 percentage point (see\cbox{blue!60!black}), federal non-defense contributed 0.17 percentage point (see\cbox{green!85!black}), and state and local government contributed 0.43 percentage point (see\cbox{purple!70!magenta}).


In [3]:
#nipa_series_codes(retrieve_table('T30100'))

### Government Summary

In [4]:
# GDP
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

# Government data Table 3.1
d = {'W021RC': 'Current receipts',
     'W054RC': 'Current tax receipts',
     'W782RC': 'Contributions for government social insurance',
     'W022RC': 'Current expenditures',
     'A955RC': 'Consumption Expenditures',
     'A084RC': 'Current transfer payments',
     'A180RC': 'Interest Payments',
     'A107RC': 'Subsidies'}
df = nipa_df(retrieve_table('T30100')['Data'], d.keys()).rename(d, axis=1)
df['Taxes'] = (df[['Current tax receipts', 
                   'Contributions for government social insurance']]
              ).sum(axis=1)
df['Other'] = (df['Current receipts'] - df['Taxes'])
df['Transfers and Subsidies'] = (df[['Current transfer payments', 'Subsidies']]
                                ).sum(axis=1)

res = df.divide(gdp, axis=0).dropna() * 100
ex = pd.concat([res.iloc[i] for i in [-1, -17]], axis=1).T.reset_index()
ex.to_csv(data_dir / 'gov_overview.csv', index_label='date')

# Text
ltdt = dtxt(ex['index'][0])['qtr1']
prdt = dtxt(ex['index'][1])['qtr1']
ltrec = value_text(ex['Current receipts'][0], 'plain')
prrec = value_text(ex['Current receipts'][1], 'plain')
ltexp = value_text(ex['Current expenditures'][0], 'plain')
prexp = value_text(ex['Current expenditures'][1], 'plain')

text = ("As an overview of the consolidated government's effect "+
        'on GDP, government '+
        f'receipts are equivalent to {ltrec} of GDP in {ltdt}, '+
        f'compared to {prrec} in {prdt}. The vast majority of '+
        'these receipts are taxes, including social insurance '+
        'contributions.\n\n Government spending is equivalent to '+
        f'{ltexp} of GDP in the latest data and {prexp} '+
        'in 2019. This includes consumption expenditures, which are '+
        "the government's operating costs, transfers and subsidies, "+
        'and interest payments. These are covered in more detail in '+
        'the following subsections.')
write_txt(text_dir / 'gov_summary.txt', text)
print(text)

As an overview of the consolidated government's effect on GDP, government receipts are equivalent to 27.0 percent of GDP in 2023 Q2, compared to 27.6 percent in 2019 Q2. The vast majority of these receipts are taxes, including social insurance contributions.

 Government spending is equivalent to 33.8 percent of GDP in the latest data and 33.2 percent in 2019. This includes consumption expenditures, which are the government's operating costs, transfers and subsidies, and interest payments. These are covered in more detail in the following subsections.


In [5]:
# Nodes and labels
s1 = (['Taxes'], 'Current receipts', 
      {'Taxes': ('blue!50!cyan', 'Taxes'), 
       'Other': ('violet', 'Other')})
s2 = (['Interest Payments', 'Transfers and Subsidies', 
      'Consumption Expenditures'], 'Current expenditures',
      {'Interest Payments': ('orange!90!red', 'Interest'), 
       'Transfers and Subsidies': 
       ('gray', 'Transfers \& Subsidies'),
       'Consumption Expenditures': ('blue!60!gray', 'Operations')})

for s in [s1, s2]:
    node = []
    n = s[1][8:11]
    for i in [0, 1]:
        # Date node
        dn = (f"\\node[below, align=center] at (axis cs: {i},0) {{"+
              f"\\scriptsize \color{{black!80}} {dtxt(ex['index'][i])['qtr3']} "+
              "\\\\[-0.22em] \\scriptsize \color{black!80} "+
              f"{dtxt(ex['index'][i])['year']}}};")
        node.append(dn)
        cs = ex[s[2].keys()].cumsum(axis=1) - (ex[s[2].keys()] / 2)
        for c in s[2].keys():
            col = 'black!90' if c == 'Transfers and Subsidies' else 'white'
            v = ex[c][i]
            hv = cs[c][i]
            tv = f'{v:.1f}'
            # Value label node
            lnode = (f'\\node[align=center] at (axis cs: {i},{hv}) '+
                    f'{{\scriptsize \color{{{col}}} {tv}}};')
            if c != 'Other':
                node.append(lnode)
            if i == 0:
                # Name label node
                nnode = (f'\\node[left, align=right, xshift=-9pt, text width=42pt] '+
                         f'at (axis cs: 0,{hv}) '+
                        f'{{\\footnotesize \color{{{s[2][c][0]}}} {s[2][c][1]}}};')
                node.append(nnode)
        tnv = ex[s[1]][i]
        tnvt = f'{tnv:.1f}'
        # Top label node (sum)
        tn = (f'\\node[above, align=center] at (axis cs: {i},{tnv}) '+
              f'{{\scriptsize \color{{black!90}} {tnvt}}};')
        node.append(tn)
    nodes = '\n'.join(node)
    write_txt(text_dir / f'gov_summary_{n}_nodes.txt', nodes)

### Government Net Investment

In [88]:
cofc = nipa_df(retrieve_table('T11000')['Data'], ['A264RC'])['A264RC']
ginv = nipa_df(retrieve_table('T30100')['Data'], ['A782RC'])['A782RC']
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

data = (((ginv - cofc) / gdp) * 100).loc['1989':]
data.name = 'Value'
data.to_csv(data_dir / 'govnetinv.csv', index_label='date')

color = 'red'
node = end_node(data, color, date='q', digits=2, offset=0.35)
write_txt(text_dir / 'govnetinv_node.txt', node)

ltdt = dtxt(data.index[-1])['qtr1']
yrdt = dtxt(data.index[-5])['qtr1']
yr2dt = dtxt(data.index[-9])['qtr1']
gni = f'\${(ginv - cofc).iloc[-1] / 1_000:.1f} billion'
gnish = f'{data.iloc[-1]:.2f} percent'
gnishyr = f'{data.iloc[-5]:.2f} percent'
gnishyr2 = f'{data.iloc[-9]:.2f} percent'

text = ("Government gross investment, less depreciation, is the government's "+
        "net investment in the tangible assets that make the economy more "+
        "productive. Government investment includes infrastructure, "+
        "buildings, equipment, intellectual property, and other capital goods. "+
        f"In the latest data, covering {ltdt}, government net investment is "+
        f"{gni}. Government net investment represents {gnish} "+
        f"of GDP in {ltdt} {c_line(color)}, compared "+
        f"to {gnishyr} in {yrdt}, and {gnishyr2} in {yr2dt}. ")
write_txt(text_dir / f'govnetinv.txt', text)
print(text)

Government gross investment, less depreciation, is the government's net investment in the tangible assets that make the economy more productive. Government investment includes infrastructure, buildings, equipment, intellectual property, and other capital goods. In the latest data, covering 2023 Q3, government net investment is \$230.7 billion. Government net investment represents 0.84 percent of GDP in 2023 Q3 (see {\color{red}\textbf{---}}), compared to 0.58 percent in 2022 Q3, and 0.67 percent in 2021 Q3. 


### Government receipts and expenditures

In [3]:
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

d = {'slggdp': {'name': 'combined state and local government', 
                'table': 'T30300', 'series': 
                {'W024RC': 'SLG_EXP', 'W023RC': 'SLG_REC'}},
     'fedgdp': {'name': 'federal government', 
                'table': 'T30200', 'series':
                {'W005RC': 'FED_REC', 'W013RC': 'FED_EXP'}}}

data = pd.DataFrame()

ltdate = dtxt(gdp.index[-1])['qtr1']

for group, details in d.items():
    df = nipa_df(retrieve_table(details['table'])['Data'], details['series'].keys())
    (df.div(gdp, axis=0) * 100).loc['1989':].to_csv(data_dir / f'{group}.csv', 
                                                    index_label='date', float_format='%g')
    
    grp = group[:3].upper()
    df.columns = [details['series'][i] for i in df.columns]
    bal = df[f'{grp}_REC'] - df[f'{grp}_EXP']
    bal_lt = bal.dropna().iloc[-1]
    bal_lt_dt = dtxt(bal.dropna().index[-1])['qtr1']
    def_sur = 'deficit' if bal_lt < 0 else 'surplus'
    df[f'{grp}_BAL'] = bal
    
    for col in df.columns:
        data[col] = df[col]
        data[col+'_GDP'] = df[col].div(gdp, axis=0) * 100
        
    bal_gdp = data[f'{grp}_BAL_GDP'].dropna().iloc[-1]
    bal_txt = (f'In {bal_lt_dt}, the {d[group]["name"]} {def_sur} was '+
               f'\${abs(bal_lt) / 1000:,.0f} billion or {abs(bal_gdp):.1f} percent of GDP. ')
    
    exp_txt = (f'{d[group]["name"].capitalize()} expenditures total '+
               f'\${data[grp+"_EXP"].iloc[-1] / 1000000:.1f} trillion, '+
               f'or {data[grp+"_EXP_GDP"].iloc[-1]:.1f} percent of GDP, in {ltdate}. ')
    
    if pd.isna(df[[i for i in df.columns if i[4:] == 'REC'][0]].iloc[-1]) == True:
        rec_txt = (f'BEA has not yet released receipts data for {ltdate}, however, '+
                   f'in {bal_lt_dt}, {d[group]["name"]} receipts total '+
                   f'\${data[grp+"_REC"].dropna().iloc[-1] / 1000000:.1f} trillion, '+
                   f'or {data[grp+"_REC_GDP"].dropna().iloc[-1]:.1f} percent of GDP. ')
    else:
        rec_txt = ('Receipts for the same period total '+
                   f'\${data[grp+"_REC"].dropna().iloc[-1] / 1000000:.1f} trillion '+
                   f'or {data[grp+"_REC_GDP"].dropna().iloc[-1]:.1f} percent of GDP. ')
    
    text = exp_txt + rec_txt + bal_txt
    write_txt(text_dir / f'{group}.txt', text)
    print(text)

Combined state and local government expenditures total \$3.8 trillion, or 13.7 percent of GDP, in 2023 Q3. BEA has not yet released receipts data for 2023 Q3, however, in 2023 Q2, combined state and local government receipts total \$3.6 trillion, or 13.2 percent of GDP. In 2023 Q2, the combined state and local government deficit was \$185 billion or 0.7 percent of GDP. 
Federal government expenditures total \$6.4 trillion, or 23.1 percent of GDP, in 2023 Q3. BEA has not yet released receipts data for 2023 Q3, however, in 2023 Q2, federal government receipts total \$4.7 trillion, or 17.3 percent of GDP. In 2023 Q2, the federal government deficit was \$1,661 billion or 6.1 percent of GDP. 


In [5]:
d = {'SLG_EXP_GDP': '\hspace{3mm} Expenditures ',
     'SLG_REC_GDP': '\hspace{3mm} Receipts ',
     'SLG_BAL_GDP': '\hspace{3mm} Surplus (+) / Deficit (-) ',
     'FED_EXP_GDP': '\hspace{3mm} Expenditures',
     'FED_REC_GDP': '\hspace{3mm} Receipts',
     'FED_BAL_GDP': '\hspace{3mm} Surplus (+) / Deficit (-)'}

cats = {'FED': ' \ \\textbf{Federal Government}', 
        'SLG': ' \ \\textbf{State \& Local Government}'}
final = pd.DataFrame()
for cat, name in cats.items():
    cols = [i for i in data.columns 
            if i.endswith('GDP') & i.startswith(cat)]
    tmp = data[cols]
    table = pd.concat([tmp.iloc[-3:].iloc[::-1].T, 
                       tmp.iloc[-5].T, 
                       tmp.iloc[-9].T, 
                       tmp.iloc[-17].T], axis=1)
    table.columns = [dtxt(i)['qtr4'] for i in table.columns]
    for i in [16, 40, 120]:
        table[f'{i/4:.0f}-year'] = tmp.iloc[-i:].mean()
    table = table.applymap('{:.1f}'.format).sort_index(ascending=False)
    blank = pd.Series(index=table.columns, dtype=float)
    blank['index'] = name
    table = table.reset_index()
    table.index = table.index + 1
    table.loc[0] = blank
    table = table.sort_index().set_index('index').rename(d)
    table.index.name = ''
    final = pd.concat([final, table]).replace('nan', '--')
final.to_csv(data_dir / 'gov_rec_exp_bal.tex', sep='&', 
             lineterminator='\\\ ', quotechar=' ')

In [51]:
#data['FED_BAL_GDP'].iloc[-13:].plot(kind='bar')

### Federal Government Current Tax Receipts

In [6]:
# Federal 
s = {'W006RC': 'Current Tax Receipts', 
     'A074RC': 'Personal Current Taxes',
     'B075RC': 'Taxes on Corporate Income', 
     'W780RC': 'Social Insurance Contributions'}
other = lambda x: x.W006RC - x.A074RC - x.B075RC
data = nipa_df(retrieve_table('T30200')['Data'], 
               s.keys()).assign(Other = other)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
sh = (data.divide(gdp, axis=0) * 100).dropna()
sh.loc['1989':].to_csv(data_dir / 'fed_gov_rec_type.csv', 
                       index_label='date')

# end nodes
srs = {'A074RC': 'green!50!black', 'B075RC': 'purple!80!violet',
       'W780RC': 'blue!60!white', 'Other': 'black!30!white'}

# date on max value
mon = {k: None for k, v in srs.items()}
mon[sh[srs.keys()].iloc[-1].idxmax()] = 'q'
       
nodes = '\n'.join([end_node(sh[n], col, date=mon[n]) 
                   for n, col in srs.items()])
write_txt(text_dir / 'fed_gov_rec_type_nodes.txt', nodes)

# Text overview
ltdt = dtxt(data.index[-1])['qtr1']
taxsrs = ['W006RC', 'W780RC']
taxval = data[taxsrs].iloc[-1].sum() / 1_000_000
taxsh = value_text(sh[taxsrs].iloc[-1].sum(), 'plain')
text = ('Taxes and social insurance contributions are the main '+
        f'federal government receipts, and total \${taxval:.1f} '+
        f'trillion in {ltdt}, equivalent to {taxsh} of GDP.')
write_txt(text_dir / 'fed_gov_rec_overview.txt', text)
print(text, '\n\n')

cl = {n: c_line(color) for n, color in srs.items()}
pctsh = value_text(sh['A074RC'].iloc[-1], 'plain')
pctlg = value_text(sh['A074RC'].mean(), 'plain')
sish = value_text(sh['W780RC'].iloc[-1], 'plain')
silg = value_text(sh['W780RC'].mean(), 'plain')
ci19 = value_text(sh.loc['2019', 'B075RC'].mean(), 'plain')
cish = value_text(sh['B075RC'].iloc[-1], 'plain')
othsh = value_text(sh['Other'].iloc[-1], 'plain')
# Text categories
text = (f'As of {ltdt}, federal personal current tax receipts '+
        f'are equivalent of {pctsh} of GDP {cl["A074RC"]}. '+
        'Some volatility in these receipts comes from swings '+
        'in yearly capital gains. Since 1989, these tax receipts '+
        f'average {pctlg} of GDP. \n\nSocial insurance '+
        'contributions are relatively stable over time and comprise '+
        f'{sish} of GDP in {ltdt} {cl["W780RC"]}, compared '+
        f'to an average of {silg} since 1989.\n\n'+
        'Corporate income tax receipts are typically two percent of '+
        'GDP during economic expansions, but were cut to '+
        f'{ci19} in 2019. In {ltdt}, these receipts '+
        f'are equivalent to {cish} of GDP {cl["B075RC"]}.\n\n'+
        'Other tax receipts, including excise taxes and customs duties '+
        f'total {othsh} of GDP in the latest data {cl["Other"]}.')
write_txt(text_dir / 'fed_gov_rec_cats.txt', text)
print(text)

Taxes and social insurance contributions are the main federal government receipts, and total \$1.8 trillion in 2023 Q3, equivalent to 16.9 percent of GDP. 


As of 2023 Q3, federal personal current tax receipts are equivalent of eight percent of GDP (see {\color{green!50!black}\textbf{---}}). Some volatility in these receipts comes from swings in yearly capital gains. Since 1989, these tax receipts average 7.9 percent of GDP. 

Social insurance contributions are relatively stable over time and comprise 6.6 percent of GDP in 2023 Q3 (see {\color{blue!60!white}\textbf{---}}), compared to an average of 6.6 percent since 1989.

Corporate income tax receipts are typically two percent of GDP during economic expansions, but were cut to one percent in 2019. In 2023 Q3, these receipts are equivalent to 1.5 percent of GDP (see {\color{purple!80!violet}\textbf{---}}).

Other tax receipts, including excise taxes and customs duties total 0.8 percent of GDP in the latest data (see {\color{black!30!w

### State & Local Government Current Tax Receipts

In [90]:
# SLG
s = {'W023RC': 'Current Receipts', 
     'B089RC': 'Federal Transfers',
     'W070RC': 'Current Tax Receipts', 
     'W071RC': 'Personal Current Taxes',
     'B245RC': 'Income Taxes',
     'B248RC': 'Sales Taxes', 
     'LA000355': 'Property Taxes'}
other = lambda x: x.W023RC - x.B089RC - x.W070RC
data = nipa_df(retrieve_table('T30300')['Data'], s.keys()).assign(Other = other)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
sh = (data.divide(gdp, axis=0) * 100).dropna()
sh.loc['1989':].to_csv(data_dir / 'slg_gov_rec_type.csv', index_label='date')

# end nodes
srs = {'W070RC': 'blue!60!black', 'B089RC': 'orange!90!red',
       'Other': 'black!30!white'}

# date on max value
mon = {k: None for k, v in srs.items()}
mon[sh[srs.keys()].iloc[-1].idxmax()] = 'q'
       
nodes = '\n'.join([end_node(sh[n], col, date=mon[n]) 
                   for n, col in srs.items()])
write_txt(text_dir / 'slg_gov_rec_type_nodes.txt', nodes)

# Nodes for small plots
cols = {'B245RC': 'blue!90!black', 
        'B248RC': 'blue!70!cyan', 
        'LA000355': 'cyan!80!white'}

for n, col in cols.items():
    write_txt(text_dir / f'slg_{n}_node.txt', end_node(sh[n], col))

In [96]:
# Text
ltdt = dtxt(sh.index[-1])['qtr1']
pryr = dtxt(sh.index[-5])['qtr1']
ltval = value_text(sh.W070RC.iloc[-1], 'plain')
prval = value_text(sh.W070RC.iloc[-5], 'plain')
fedsh = value_text(sh.B089RC.iloc[-1], 'plain')
fedpy = value_text(sh.B089RC.iloc[-5], 'plain')
fed89 = value_text(sh.loc['1989', 'B089RC'].mean(), 'plain')
othsh = value_text(sh.Other.iloc[-1], 'plain')
ch19 = value_text(sh.W070RC.iloc[-1] - 
                  sh.loc['2019', 'W070RC'].mean(), 'increase_by')
itch19 = value_text(sh.B245RC.iloc[-1] - 
                    sh.loc['2019', 'B245RC'].mean(), 'increase_by')
cl = {n: c_line(color) for n, color in {**cols, **srs}.items()}
text = (f'In {ltdt}, combined state and local government tax receipts '+
        f'total {ltval} of GDP {cl["W070RC"]}, following '+
        f'{prval} of GDP one year prior. Since 2019, these tax receipts '+
        f'{ch19} of GDP. State and local government income tax '+
        f'receipts {itch19} of GDP over the same period.\n\n'+
        'Federal government transfers to state and local governments total '+
        f'{fedsh} of GDP in {ltdt} {cl["B089RC"]}, and {fedpy} one year prior. '+
        'These transfers peaked during COVID-19 relief efforts, but have been '+
        f'climbing over time, from {fed89} of GDP in 1989. '+
        f'Other receipts are equivalent to {othsh} of GDP in {ltdt} {cl["Other"]}. ')
write_txt(text_dir / 'slg_gov_rec_cats.txt', text)
print(text)

In 2023 Q2, combined state and local government tax receipts total 8.6 percent of GDP (see {\color{blue!60!black}\textbf{---}}), following 9.4 percent of GDP one year prior. Since 2019, these tax receipts decreased by 0.4 percent of GDP. State and local government income tax receipts decreased by 0.3 percent of GDP over the same period.

Federal government transfers to state and local governments total 3.6 percent of GDP in 2023 Q2 (see {\color{orange!90!red}\textbf{---}}), and 3.8 percent one year prior. These transfers peaked during COVID-19 relief efforts, but have been climbing over time, from 1.6 percent of GDP in 1989. Other receipts are equivalent to 1.1 percent of GDP in 2023 Q2 (see {\color{black!30!white}\textbf{---}}). 


### Government Receipts Overview

In [97]:
s = ['W021RC', 'W054RC', 'W782RC']
taxsi = lambda x: x.W054RC + x.W782RC
data = nipa_df(retrieve_table('T30100')['Data'], s).assign(TAXSI = taxsi).dropna()

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
sh = (data.divide(gdp, axis=0) * 100).dropna()
sh.loc['1989':].to_csv(data_dir / 'gov_rec_overview.csv', index_label='date')

# end nodes
srs = {'W021RC': 'cyan!65!white', 'TAXSI': 'blue!50!violet'}

# date on max value
maxkey = sh[srs.keys()].iloc[-1].idxmax()
mon = {k: None for k, v in srs.items()}
mon[maxkey] = 'q'
off = {k: 0 for k, v in srs.items()}
off[maxkey] = 0.35
       
nodes = '\n'.join([end_node(sh[n], col, date=mon[n], offset=off[n]) 
                   for n, col in srs.items()])
write_txt(text_dir / 'gov_rec_overview_nodes.txt', nodes)

# Text
ltdt = dtxt(sh.index[-1])['qtr1']
rec_tot = data.W021RC.iloc[-1] / 1_000_000
cl = {n: c_line(color) for n, color in srs.items()}
totsh = value_text(sh.W021RC.iloc[-1], 'plain')
taxsh = value_text(sh.TAXSI.iloc[-1], 'plain')
taxcomp = value_text((data.TAXSI.iloc[-1] / 
                      data.W021RC.iloc[-1]) * 100, 'plain')

text = (f'Government receipts total \${rec_tot:.1f} trillion in {ltdt}, '+
        f'representing {totsh} of GDP {cl["W021RC"]}. Taxes and social '+
        f'insurance contributions comprise {taxcomp} of receipts and are '+
        f'equivalent to {taxsh} of GDP in {ltdt} {cl["TAXSI"]}. ')
write_txt(text_dir / 'gov_rec_overview.txt', text)
print(text)

Government receipts total \$7.3 trillion in 2023 Q2, representing 27.0 percent of GDP (see {\color{cyan!65!white}\textbf{---}}). Taxes and social insurance contributions comprise 94.8 percent of receipts and are equivalent to 25.6 percent of GDP in 2023 Q2 (see {\color{blue!50!violet}\textbf{---}}). 


In [56]:
#nipa_series_codes(retrieve_table('T30300'))

### Government Spending Overview

In [101]:
srs = {'T30100': {'W022RC': 'Current Expenditures'},
       'T30200': {'W013RC': 'Federal Current Expenditures',
                  'B089RC': 'SLG Grants'},
       'T30300': {'W024RC': 'SLG Current Expenditures'}}

df = pd.concat([nipa_df(retrieve_table(t)['Data'], d.keys()).rename(d, axis=1) 
               for (t, d) in srs.items()], axis=1)
df['Federal'] = df['Federal Current Expenditures'] - df['SLG Grants']
df['SLG'] = df['SLG Current Expenditures'] - df['SLG Grants']
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
sh = (df.divide(gdp, axis=0) * 100)
sh.loc['1989':].to_csv(data_dir / 'gov_exp_overview.csv', 
                       index_label='date')

# End nodes
color = 'black!80!white'
node = end_node(sh['Current Expenditures'], color, date='q', offset=0.35)
write_txt(text_dir / 'gov_exp_tot_node.txt', node)

# Nodes for latest values
cols = ['Federal', 'SLG Grants', 'SLG']
sdf = sh[cols].iloc[-1]
height = ((sdf.cumsum() - (sdf / 2) + 2)).to_dict()
val = sdf.to_dict()
dtp = dtxt(sh.index[-1] + pd.DateOffset(months=2))['datetime']
nodes = [f'\\absnode{{{{{dtp}}}}}{{{height[i]}}}{{\scriptsize {val[i]:.1f}}}' 
         for i in cols]
nodetext = '\n'.join(nodes)
write_txt(text_dir / 'gov_exp_overview_nodes.txt', nodetext)

# Summary Text main
s = 'Current Expenditures'
ltdt = dtxt(sh.index[-1])['qtr1']
pydt = dtxt(sh.index[-5])['qtr1']
totval = df[s].iloc[-1] / 1_000_000
totsh = value_text(sh[s].iloc[-1], 'plain')
totshpy = value_text(sh[s].iloc[-5], 'plain')
totshcov = value_text(sh.loc['2020': '2021', s].mean(), 'plain')
totshlt = value_text(sh.loc['1989':, s].mean(), 'plain')

cl = c_line('black!80!white')

text = (f'As of {ltdt}, government current expenditures total \${totval:.1f} '+
        f'trillion, which is {totsh} of GDP {cl}. One year prior, '+
        f'in {pydt}, government current expenditures total {totshpy} '+
        'of GDP. Government spending peaked during the COVID-19 pandemic, averaging '+
        f'{totshcov} of GDP from 2020 to 2021. Since 1989, government spending '+
        f'averages {totshlt} of GDP.')
write_txt(text_dir / 'gov_exp_overview.txt', text)
print(text)

As of 2023 Q3, government current expenditures total \$9.3 trillion, which is 33.5 percent of GDP (see {\color{black!80!white}\textbf{---}}). One year prior, in 2022 Q3, government current expenditures total 33.6 percent of GDP. Government spending peaked during the COVID-19 pandemic, averaging 40.9 percent of GDP from 2020 to 2021. Since 1989, government spending averages 33.3 percent of GDP.


In [102]:
#nipa_series_codes(retrieve_table('T30100'))

In [103]:
# Table 
s = {'W022RC': f'Current Expenditures {c_line("black!80!white", see=False)}',
     'A955RC': '\hspace{2mm}Consumption Expenditures',
     'A084RC': '\hspace{2mm}Current Transfer Payments',
     'A180RC': '\hspace{2mm}Interest Payments',
     'A107RC': '\hspace{2mm}Subsidies'}
df = nipa_df(retrieve_table('T30100')['Data'], s)
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
res = (df.divide(gdp, axis=0) * 100)
data = res.iloc[-3:].iloc[::-1].T
data[res.index[-5]] = res.iloc[-5]
data[res.index[-9]] = res.iloc[-9]
cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]

data.columns = cols
data['1-year'] = res.rolling(4).mean().iloc[-1]
data['10-year'] = res.rolling(40).mean().iloc[-1]
data['30-year'] = res.rolling(120).mean().iloc[-1]
data.index = data.index.map(s)
data = data.applymap('{:.1f}'.format)
data.to_csv(data_dir / 'gov_exp.tex', sep='&', lineterminator='\\\ ', quotechar=' ')

In [105]:
# text
ltdt = dtxt(res.index[-1])['qtr1']
pydt = dtxt(res.index[-5])['qtr1']
fedsh = value_text(sh.Federal.iloc[-1], 'plain')
fedpy = value_text(sh.Federal.iloc[-5], 'plain')
fedtr = value_text(sh['SLG Grants'].iloc[-1], 'plain')
slgsh = value_text(sh.SLG.iloc[-1], 'plain')
cbfed = c_box('blue!65!gray!90!white')
cbtr = c_box('cyan!40!white')
cbslg = c_box('red!20!purple!70!white')
consh = value_text(res.A955RC.iloc[-1], 'plain')
conpy = value_text(res.A955RC.iloc[-5], 'plain')
conlg = value_text(res.A955RC.iloc[-121:].mean(), 'plain')
trsh = value_text(res.A084RC.iloc[-1], 'plain')
trpy = value_text(res.A084RC.iloc[-5], 'plain')
trlg = value_text(res.A084RC.iloc[-121:].mean(), 'plain')
intsh = value_text(res.A180RC.iloc[-1], 'plain')
intpy = value_text(res.A180RC.iloc[-5], 'plain')
intlg = value_text(res.A180RC.iloc[-121:].mean(), 'plain')
subsh = value_text(res.A107RC.iloc[-1], 'plain')
subpy = value_text(res.A107RC.iloc[-5], 'plain')
text = ('By level of government, federal current expenditures are equivalent '+
        f'to {fedsh} of GDP in {ltdt}, and {fedpy} in '+
        f'{pydt} {cbfed}. Federal government transfers to state '+
        f'and local governments comprise {fedtr} of GDP in the '+
        f'latest data {cbtr}. State and local government current '+
        'expenditures, excluding transfers from the federal government, '+
        f'are equivalent to {slgsh} of GDP in {ltdt} {cbslg}. \n\nBy '+
        'category, consumption expenditures represent '+
        f'{consh} of GDP in {ltdt}, and {conpy} '+
        'of GDP one year prior. Over the past 30 years, consumption '+
        f'expenditures average {conlg} of GDP. Current '+
        'transfer payments, which are largely government social benefits, '+
        f'total {trsh} of GDP in {ltdt}, compared to '+
        f'{trpy} one year prior, and a long-term average '+
        f'of {trlg}.\n\nConsolidated government interest '+
        f'payments are {intsh} of GDP in the '+
        f'latest data, compared to {intpy} one year prior. '+
        f'Interest payments comprise {intlg} over the past '+
        '30 years, on average. Government subsidies to businesses total '+
        f'{subsh} of GDP in {ltdt} and {subpy} in '+
        f'{pydt}. ')
write_txt(text_dir / 'gov_exp_overview2.txt', text)
print(text)

By level of government, federal current expenditures are equivalent to 19.8 percent of GDP in 2023 Q3, and 19.7 percent in 2022 Q3 (see\cbox{blue!65!gray!90!white}). Federal government transfers to state and local governments comprise 3.3 percent of GDP in the latest data (see\cbox{cyan!40!white}). State and local government current expenditures, excluding transfers from the federal government, are equivalent to 10.4 percent of GDP in 2023 Q3 (see\cbox{red!20!purple!70!white}). 

By category, consumption expenditures represent 13.7 percent of GDP in 2023 Q3, and 13.8 percent of GDP one year prior. Over the past 30 years, consumption expenditures average 14.8 percent of GDP. Current transfer payments, which are largely government social benefits, total 14.9 percent of GDP in 2023 Q3, compared to 15.5 percent one year prior, and a long-term average of 13.5 percent.

Consolidated government interest payments are 4.6 percent of GDP in the latest data, compared to 3.8 percent one year prior

### Personal Income

See also table in generate_table notebook

In [58]:
# deflator
d = nipa_df(retrieve_table('T20304')['Data'], ['DPCERG'])['DPCERG']
deflator = d.iloc[-1] / d

# collect and combine series
s = ['A065RC', 'A033RC', 'A041RC', 'A048RC', 'W210RC', 'A577RC', 'A061RC']
df = (nipa_df(retrieve_table('T20100')['Data'], s)
      .assign(CAPITAL = lambda x: x['A041RC'] + x['A048RC'] + x['W210RC'],
              TRANSFER = lambda x: x['A577RC'] - x['A061RC'])
      .drop(['A061RC', 'A041RC', 'A048RC', 'W210RC', 'A577RC'], axis=1)
      .multiply(deflator, axis=0))
growth_contrib_ann(df, 'A065RC').loc['1989':].to_csv(data_dir / 'pi.csv', index_label='date')

data = growth_contrib_ann(df, 'A065RC').rename({'A065RC': 'TOTAL', 'A033RC': 'LABOR'}, axis=1)

ltdate = dtxt(data.index[-1])['qtr1']
tot = value_text(data['TOTAL'].iloc[-1], digits=2)
slist = ['LABOR', 'CAPITAL', 'TRANSFER']
style = {k: 'contribution' for k in slist}
style['LABOR'] = 'contribution_to'
d = {}
for i in slist:
    d[i] = value_text(data[i].iloc[-1], style=style[i], 
                      ptype='pp', digits=2)
text = (f'Aggregate real personal income {tot} over the year ending {ltdate}. '+
        f'Labor income {d["LABOR"]} overall growth, capital '+
        f'income {d["CAPITAL"]}, and net government benefits {d["TRANSFER"]}. ')
write_txt(text_dir / 'pi.txt', text)
print(text)

Aggregate real personal income increased 1.36 percent over the year ending 2023 Q3. Labor income contributed 1.25 percentage points to overall growth, capital income contributed 0.35 percentage point, and net government benefits subtracted 0.25 percentage point. 


### Personal Income Table

In [ ]:
series = {
 'A065RC': '\hspace{0.5mm}Personal Income (Pre-Tax Income)',
 'A033RC': '\hspace{-2mm}\cbox{green!55!blue!72!black}Labor',
 'A034RC': '\hspace{3mm} Wages \& Salaries',
 'A038RC': '\hspace{3mm} Supplements to Wages \& Salaries',
 'CAPITAL': '\hspace{-2mm}\cbox{yellow!85!orange!88!white}Capital',
 'A041RC': "\hspace{3mm} Proprietors' Income",
 'A048RC': '\hspace{3mm} Rental Income',
 'W210RC': '\hspace{3mm} Personal Income Receipts on Assets',
 'A064RC': '\hspace{3mm} Personal Interest Income',
 'B703RC': '\hspace{3mm} Personal Dividend Income',
 'TRANSFER': '\hspace{-2mm}\cbox{cyan!90!blue!90!white}Net Government Benefits',
 'A577RC': 'Personal current transfer receipts',
 'A063RC': '\hspace{2mm} Government Social Benefits',
 'W823RC': '\hspace{3mm} Social Security',
 'W824RC': '\hspace{3mm} Medicare',
 'W729RC': '\hspace{3mm} Medicaid',
 'W825RC': '\hspace{3mm} Unemployment Insurance',
 'W826RC': "\hspace{3mm} Veterans' Benefits",
 'W827RC': '\hspace{3mm} Other',
 'A061RC': '\hspace{2mm} Less: Social Insurance Contributions'}

# PCE price deflator
pce = nipa_df(retrieve_table('T20304')['Data'], ['DPCERG'])['DPCERG']
deflator = d.iloc[-1] / d

# Calculate real growth
s = [key for key, value in series.items() 
     if key not in ['CAPITAL', 'TRANSFER']]
df = (nipa_df(retrieve_table('T20100')['Data'], s)
      .assign(CAPITAL = lambda x: x['A041RC'] + x['A048RC'] + x['W210RC'],
              TRANSFER = lambda x: x['A577RC'] - x['A061RC'],
              A061RC = lambda x: - x['A061RC'])
      .drop(['W210RC', 'A577RC'], axis=1)
      .multiply(deflator, axis=0))
result = growth_contrib_ann(df, 'A065RC')
data = result.iloc[-5:].iloc[::-1].T
data.columns = [f' {q.year} Q{q.quarter}' 
                if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
                for i, q in enumerate(data.columns)]

# Averages over past year, 10 years, and 30 years
for i in 4, 40, 120:
    data[f'{i/4}-year'] = result.rolling(i).mean().iloc[-1]
# Save table    
data.index = data.index.map(series)
(data.reindex(list(series.values())).dropna().round(2).applymap('{:,.2f}'.format)
     .to_csv(data_dir / 'pi.tex', sep='&', lineterminator='\\\ ', quotechar=' '))

### Government Personal Income

In [59]:
s = ['DPCERG']

d = nipa_df(retrieve_table('T20304')['Data'], s)['DPCERG']
deflator = d.iloc[-1] / d

s = ['B230RC']

population = nipa_df(retrieve_table('T20100')['Data'], s)['B230RC']

s = ['A063RC', 'B202RC', 'A061RC', 'W055RC']
df = nipa_df(retrieve_table('T20100')['Data'], s)

n = {'A063RC': 'Welfare',
     'B202RC': 'WandS'}
result = df.multiply(deflator, axis=0).divide(population, axis=0)
result['TaxSI'] = result['A061RC'] + result['W055RC']
result = result.drop(['A061RC', 'W055RC'], axis=1).rename(n, axis=1)
result.loc['1989':].to_csv(data_dir / 'govpi.csv', index_label='date')

ltdate = dtxt(result.index[-1])['qtr1']
prdt = '2019-10-01'
prdate = dtxt(pd.to_datetime(prdt))['qtr1']
wsltval = result['WandS'].iloc[-1] * 1_000
wsprval = result.loc[prdt, 'WandS'] * 1_000
wltval = result['Welfare'].iloc[-1] * 1_000
wprval = result.loc[prdt, 'Welfare'] * 1_000
w89val = result.loc['1989-01-01', 'Welfare'] * 1_000
tltval = result['TaxSI'].iloc[-1] * 1_000
tprval = result.loc[prdt, 'TaxSI'] * 1_000
t89val = result.loc['1989-01-01', 'TaxSI'] * 1_000

text = (f'In {ltdate}, government worker wages and salaries '+
        f'were equivalent to \${wsltval:,.0f} per capita, following a '+
        f'price-adjusted \${wsprval:,.0f} in {prdate} '+
        '(see {\color{orange}\\textbf{---}}). Net government benefits were equivalent '+
        f'to \${wltval:,.0f} per capita in {ltdate}, compared to \${wprval:,.0f} '+
        f'per capita in {prdate} '+
        '(see {\color{violet}\\textbf{---}}). In 1989 Q1, net benefits were '+
        f'equivalent to \${w89val:,.0f} per person.  \n\n Personal current taxes '+
        f'and social insurance contributions total \${tltval:,.0f} per capita '+
        f'in {ltdate}, \${tprval:,.0f} in {prdate}, and \${t89val:,.0f} in 1989 '+
        '(see {\color{cyan!80!blue}\\textbf{---}}).')
write_txt(text_dir / 'govpi.txt', text)
print(text)

In 2023 Q3, government worker wages and salaries were equivalent to \$5,203 per capita, following a price-adjusted \$5,199 in 2019 Q4 (see {\color{orange}\textbf{---}}). Net government benefits were equivalent to \$11,886 per capita in 2023 Q3, compared to \$10,940 per capita in 2019 Q4 (see {\color{violet}\textbf{---}}). In 1989 Q1, net benefits were equivalent to \$4,420 per person.  

 Personal current taxes and social insurance contributions total \$13,791 per capita in 2023 Q3, \$12,849 in 2019 Q4, and \$8,096 in 1989 (see {\color{cyan!80!blue}\textbf{---}}).


### Government Consumption and Investment

In [60]:
s = ['A823RC', 'A829RC', 'A191RC', 'A824RC', 'A825RC']
df = nipa_df(retrieve_table('T10105')['Data'], s).sort_index()
result = df.drop('A191RC', axis=1).divide(df['A191RC'], axis=0) * 100
result.loc['1989':].to_csv(data_dir / 'govci.csv', index_label='date')

ltdate = dtxt(result.index[-1])['qtr1']
prdt = pd.to_datetime('2019-10-01')
prdate = dtxt(prdt)['qtr1']
fndnom = df['A825RC'].iloc[-1] / 1_000
fndlt = result['A825RC'].iloc[-1]
fndpr = result.loc[prdt, 'A825RC']
dlt = result['A824RC'].iloc[-1]
dpr = result.loc[prdt, 'A824RC']
d89 = result.loc['1989-01-01', 'A824RC']
slt = result['A829RC'].iloc[-1]
spr = result.loc[prdt, 'A829RC']

text = (f'In {ltdate}, federal non-defense spending and investment '+
        f'was \${fndnom:,.1f} billion, equivalent to {fndlt:.1f} percent '+
        'of GDP (see {\color{green!85!black}\\textbf{---}}), compared '+
        f'to {fndpr:.1f} percent of GDP in {prdate}. Federal spending '+
        f'on national defense was equivalent to {dlt:.1f} percent of '+
        f'GDP in the latest quarter and {dpr:.1f} percent in {prdate} '+
        '(see {\color{blue!60!black}\\textbf{---}}). National defense '
        f'spending was {d89:.1f} percent of GDP in 1989 Q1. \n\n In '+
        f'{ltdate}, state and local government spending and investment '+
        f'was equivalent to {slt:.1f} percent of GDP, compared to '+
        f'{spr:.1f} percent in {prdate} '+
        '(see {\color{purple!70!magenta}\\textbf{---}}).')
write_txt(text_dir / 'govci.txt', text)
print(text)

In 2023 Q3, federal non-defense spending and investment was \$782.1 billion, equivalent to 2.8 percent of GDP (see {\color{green!85!black}\textbf{---}}), compared to 2.6 percent of GDP in 2019 Q4. Federal spending on national defense was equivalent to 3.6 percent of GDP in the latest quarter and 4.0 percent in 2019 Q4 (see {\color{blue!60!black}\textbf{---}}). National defense spending was 6.9 percent of GDP in 1989 Q1. 

 In 2023 Q3, state and local government spending and investment was equivalent to 10.8 percent of GDP, compared to 11.1 percent in 2019 Q4 (see {\color{purple!70!magenta}\textbf{---}}).


### Consumer Spending Growth Contributions

In [14]:
s = 'B230RC'
pop = nipa_df(retrieve_table('T20100')['Data'], [s])[s]
s = 'DPCERG'
d = nipa_df(retrieve_table('T20304')['Data'], [s])[s]
deflator = d.iloc[-1] / d

s = ['A067RC', 'A068RC', 'A071RC', 'DPCERC']

df = (nipa_df(retrieve_table('T20100')['Data'], s)
      .assign(OTHER = lambda x: -(x['A068RC'] - x['DPCERC']),
              SAVING = lambda x: -x['A071RC'])
      .rename({'A067RC': 'INCOME', 'DPCERC': 'SPENDING'}, axis=1)
      .drop(['A068RC', 'A071RC'], axis=1)
      .divide(pop, axis=0)
      .multiply(deflator, axis=0))

data = growth_contrib(df, 'SPENDING').rolling(4).mean()
data.loc['1989':].to_csv(data_dir / 'pcedecomp.csv', index_label='date')

In [64]:
# Long-term text 
n = (df.index[-1] - df.index[0]).days / 365.25
chtot = (((((df.iloc[-1] - df.iloc[0]) / df.SPENDING.iloc[0]) + 1)
          ** (1/n)) - 1) * 100
expch = value_text(chtot.SPENDING, 'plain')
incch = value_text(chtot.INCOME, 'plain')
sav = ('Reduced saving' if chtot.SAVING > 0.1 
       else 'Increased saving' if chtot.SAVING < -0.1 
       else 'Saving')
savch = (value_text(chtot.SAVING, threshold=0.1) if abs(chtot.SAVING) < 0.1 
         else value_text(chtot.SAVING, 'contribution', ptype='pp', 
                         casual=True))
oth = ('increases in other outlays' if chtot.OTHER < -0.1 
       else 'decreases in other outlays' if ch.OTHER > 0.1 
       else 'other outlays')
othch = value_text(chtot.OTHER, 'contribution', threshold=0.1, 
                   ptype='pp', obj='plural')

text = ('Since 1989, annualized real per capita consumer spending '+
        f'growth of {expch} is explained by a {incch} increase '+
        f'in disposable income. '+
        f'{sav} {savch}, while {oth} {othch} per year.')

# Recent data text
date = f'{data.index[-1].year} Q{data.index[-1].quarter}'
pcetext = value_text(data['SPENDING'].iloc[-1], adj='average')

slist = ['INCOME', 'SAVING', 'OTHER']
d = {}
for i in slist:
    val = data[i].iloc[-1]
    vt = value_text(data[i].iloc[-1], style='contribution', 
           ptype='pp', casual=True)
    if f'{abs(val):.1f}' == '0.0':
        vt = "didn't affect the total"
    d[i] = vt
pce19 = value_text(data.loc['2019', 'SPENDING'].mean(), adj='average')
dpi19 = value_text(data.loc['2019', 'INCOME'].mean(), style='contribution', 
           ptype='pp')
save19 = value_text(data.loc['2019', 'SAVING'].mean(), style='contribution', 
           ptype='pp')
inc = data.INCOME.iloc[-1]
chinc = 'Higher' if inc > 0.1 else 'Lower' if inc < -0.1 else 'Flat'
ltsv = data['SAVING'].iloc[-1].round(2)
svid = 'decreased' if ltsv > 0 else '' if ltsv == 0 else 'increased'
ltot = data['OTHER'].iloc[-1].round(2)
otid = 'decreases in' if ltsv > 0 else '' if ltsv == 0 else 'increases in'
pcetxt = (f'{text}\n\n Spending {pcetext} over the '+
          f'four quarters ending {date}. {chinc} income '+
          f'{d["INCOME"]}, {svid} saving {d["SAVING"]}, and '+
          f'{otid} other outlays {d["OTHER"]}.')
write_txt(text_dir / 'pcedecomp.txt', pcetxt)
print(pcetxt)

Since 1989, annualized real per capita consumer spending growth of 1.8 percent is explained by a 1.8 percent increase in disposable income. Saving was virtually unchanged, while increases in other outlays subtracted 0.1 percentage point per year.

 Spending increased at an average rate of 1.9 percent over the four quarters ending 2023 Q3. Higher income added 3.5 percentage points, increased saving subtracted 0.8 percentage point, and increases in other outlays subtracted 0.8 percentage point.


### Nominal Consumer Spending Growth by Year

In [139]:
df = (nipa_df(retrieve_table('T20100')['Data'], ['DPCERC'])
      .rename({'DPCERC': 'NOMINAL'}, axis=1))
s = 'B230RC'
df['POP'] = nipa_df(retrieve_table('T20100')['Data'], [s])[s]
s = 'DPCERG'
defl = nipa_df(retrieve_table('T20304')['Data'], [s])[s]
df['INFL'] = (defl[-1] / defl)
df['REAL'] = df['NOMINAL'] * df['INFL']
df['REALPC'] = df['REAL'] / df['POP']
dfc = df.groupby(df.index.year).filter(lambda x: len(x) == 4)
res = (dfc.resample('AS').mean().pct_change() * 100).dropna()
res['INFL_CONTR'] = res['NOMINAL'] - res['REAL']
res['POP_CONTR'] = res['REAL'] - res['REALPC']
keep_col = ['POP_CONTR', 'INFL_CONTR', 'REALPC']
res[keep_col].to_csv(data_dir / 'npcedecomp.csv', index_label='date')
res.index = res.index + pd.DateOffset(months=7)
res['NOMINAL'].to_csv(data_dir / 'npce_gr.csv', index_label='date')
node = end_node(res.NOMINAL, 'black', date='y', size=1.4, offset=0.18, 
                xoffset=0.05)
write_txt(text_dir / 'npce_gr_node.txt', node)

In [164]:
nom89 = cagr(dfc.loc['1989':, 'NOMINAL'])
real89 = cagr(dfc.loc['1989':, 'REAL'])
rpc89 = cagr(dfc.loc['1989':, 'REALPC'])
nom89t = value_text(nom89, 'plain')
rpc89t = value_text(rpc89, 'plain')
inf89t = value_text(nom89-real89, 'plain', ptype='pp')
pop89t = value_text(real89-rpc89, 'plain', ptype='pp')
nomlt = value_text(res['NOMINAL'].iloc[-1], 'plain')
rpclt = value_text(res['REALPC'].iloc[-1], 'plain')
inflt = value_text(res['INFL_CONTR'].iloc[-1], 'plain')
poplt = value_text(res['POP_CONTR'].iloc[-1], 'plain')
ltyr = res.index[-1].year
text = ('Since 1989, total US consumer spending increased at an annual rate '+
        f'of {nom89t}. Real per capita spending increased by {rpc89t} per year, '+
        f'inflation added {inf89t}, and population growth added {pop89t}. '+
        f'In the latest full year of data, {ltyr}, nominal consumer spending growth '+
        f'is {nomlt}, real per capita growth is {rpclt}, inflation is {inflt}, '+
        f'and population growth is {poplt}.')
write_txt(text_dir / 'npce_gr_decomp.txt', text)
print(text)

Since 1989, total US consumer spending increased at an annual rate of five percent. Real per capita spending increased by 1.8 percent per year, inflation added 2.3 percentage points, and population growth added 0.9 percentage point. In the latest full year of data, 2022, nominal consumer spending growth is 9.2 percent, real per capita growth is 2.2 percent, inflation is 6.6 percent, and population growth is 0.4 percent.


### Sectoral Accounts

**NOTE:** Need to convert "deficit", "borrower" etc to parameters!

In [63]:
def sect_txt(val):
    lb = ('borrower', 'deficit') if val < 0 else ('lender', 'surplus')
    d = {'v': val,
         'vt': f'{val:.1f}',
         'avt': f'{abs(val):.1f}',
         'vg': f'the equivalent of {abs(val):.1f} percent of GDP',
         'lb': lb[0],
         'ds': lb[1]}
    return d

In [64]:
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
s = ['W162RC', 'W994RC', 'AD01RC', 'W995RC', 'W996RC', 'AD03RC']
sb = nipa_df(retrieve_table('T50100')['Data'], s)
df = (sb.div(gdp, axis=0) * 100).dropna()
res = df[['W995RC', 'W996RC', 'AD03RC']].loc['1989':]
res.to_csv(data_dir / 'sectbal2.csv', index_label='date')

data = pd.DataFrame()
data['PRIV'] = df['W994RC']
data['GOV'] = df['AD01RC']
data['ROW'] = -df['W162RC'] # Negative in raw data
data = data.dropna().loc['1989':]
data.to_csv(data_dir / 'sectbal.csv', index_label='date')

date = dtxt(data.index[-1])['qtr1']
pcdt = '2019-10-01'

priv = sect_txt(data.PRIV.iloc[-1])
privpr = sect_txt(data.loc['2019', 'PRIV'].mean())
row = sect_txt(data.ROW.iloc[-1])
rowpr = sect_txt(data.loc['2019', 'ROW'].mean())
gov = sect_txt(data.GOV.iloc[-1])
govpr = sect_txt(data.loc['2019', 'GOV'].mean())

compare = compare_text(priv['v'], privpr['v'], [0.4, 1.0, 3.5])

txt = (f"In {date}, the US private sector was a net {priv['lb']} "+
       f"(running a {priv['ds']}) of {priv['vg']}, {compare} the "+
       f"{privpr['avt']} percent surplus in 2019. The rest of the world "+
       f"was a net {row['lb']} to the US to {row['vg']} in {date}, compared "+
       f"to {rowpr['avt']} percent in 2019. Balancing these transactions, "+
       "the government (federal, state, and local combined) was a net "+
       f"{gov['lb']} (running a {gov['ds']}) of {gov['vg']} in {date}, "+
       f"compared to {govpr['avt']} percent in 2019. ")
write_txt(text_dir / 'sectbal.txt', txt)
print(txt, '\n')

date = dtxt(df.index[-1])['qtr1']
hh = sect_txt(df.W996RC.iloc[-1])
hhpr = sect_txt(df.loc['2019', 'W996RC'].mean())
pb = sect_txt(df.W995RC.iloc[-1])
pbpr = sect_txt(df.loc['2019', 'W995RC'].mean())   
text = ('Breaking out the two main categories in the private sector, households '+
       f'were net {hh["lb"]}s (ran a {hh["ds"]}) of {hh["vg"]} in {date} '+
        '(see\cbox{orange!90!yellow}), while private businesses--corporate '+
       f'and noncorporate--were net {pb["lb"]}s of {pb["vg"]} '+
       '(see\cbox{purple!50!red}). In 2019, households were net '+
       f'{hhpr["lb"]}s of {hhpr["avt"]} percent, and private businesses were net '+
       f'{pbpr["lb"]}s of {pbpr["avt"]} percent.') 
write_txt(text_dir / 'sectbal2.txt', text)
print(text)

In 2023 Q2, the US private sector was a net lender (running a surplus) of the equivalent of 4.6 percent of GDP, in line with the 4.6 percent surplus in 2019. The rest of the world was a net lender to the US to the equivalent of 3.1 percent of GDP in 2023 Q2, compared to 2.1 percent in 2019. Balancing these transactions, the government (federal, state, and local combined) was a net borrower (running a deficit) of the equivalent of 7.7 percent of GDP in 2023 Q2, compared to 6.7 percent in 2019.  

Breaking out the two main categories in the private sector, households were net lenders (ran a surplus) of the equivalent of 2.7 percent of GDP in 2023 Q2 (see\cbox{orange!90!yellow}), while private businesses--corporate and noncorporate--were net lenders of the equivalent of 1.9 percent of GDP (see\cbox{purple!50!red}). In 2019, households were net lenders of 4.0 percent, and private businesses were net lenders of 0.6 percent.


### Current Account Balance

In [65]:
s = ['A191RC']

gdp = nipa_df(retrieve_table('T10105')['Data'], s)

n = {'A124RC': 'Current Account Balance',
     'A1073C': 'Current Receipts',
     'B020RC': '\hspace{1mm}Exports',
     'A253RC': '\hspace{3mm}Goods',
     'A332RC': '\hspace{5mm}Durable',
     'A339RC': '\hspace{5mm}Non-Durable',
     'A646RC': '\hspace{3mm}Services',
     'B645RC': '\hspace{1mm}Income Receipts',
     'LA000035': '\hspace{1mm}Transfer Receipts',
     'W163RC': 'Current payments',
     'B021RC': '\hspace{1mm}Imports',
     'A255RC': '\hspace{3mm}Goods',
     'A333RC': '\hspace{5mm}Durable',
     'A340RC': '\hspace{5mm}Non-Durable',
     'B656RC': '\hspace{3mm}Services',
     'A655RC': '\hspace{1mm}Income Payments',
     'A123RC': '\hspace{1mm}Transfer Payments'}

s = ['A124RC', 'GOODS', 'SERVICES', 'INCOME', 'TRANSFERS']

df = (nipa_df(retrieve_table('T40100')['Data'], n.keys())
      .assign(GOODS = lambda x: x['A253RC'] - x['A255RC'],
              SERVICES = lambda x: x['A646RC'] - x['B656RC'],
              INCOME = lambda x: x['B645RC'] - x['A655RC'],
              TRANSFERS = lambda x: - x['A123RC']))

data = (df.div(nipa_df(retrieve_table('T10105')['Data'], ['A191RC']
               )['A191RC'], axis=0).loc['1989':].multiply(100).round(2))

data.loc['1989':, s].dropna().to_csv(data_dir / 'cab.csv', index_label='date')

node = end_node(data['A124RC'].dropna(), 'black', date='q')
write_txt(text_dir / 'cab_node.txt', node)  

cab = abs(data.dropna()['A124RC'].iloc[-1])
tb = abs(data.dropna()['GOODS'].iloc[-1])
ld = dtxt(data.dropna().index[-1])['qtr1']
ltdate = dtxt(data.index[-1])['qtr1']
prdate = dtxt(data.index[-2])['qtr1']
lttbval = abs(data['GOODS'].iloc[-1])
tbprval = abs(data['GOODS'].iloc[-2])
prval = abs(data['A124RC'].iloc[-2])

if pd.isna(data['A124RC'].iloc[-1]) == True:
    mtxt = (f'The initial GDP report for {ltdate} does not include the '+
            'data needed to calculate the current account balance, however, '+
            f'the goods trade deficit for {ltdate} is equivalent to '+
            f'{lttbval:.1f} percent of GDP. ')
else:
    mtxt = (f'In {prdate}, the current account deficit was equivalent to '
            f'{prval:.1f}, and the trade deficit was equivalent to '+
            f'{tbprval:.1f} percent of GDP.')

text = (f'As of {ld}, the US runs a current account deficit of {cab:.1f} '+
        'percent of GDP, primarily as the result of a trade deficit on '+
        f'goods of {tb:.1f} percent of GDP. {mtxt}')
write_txt(text_dir / 'cab.txt', text)
print(text)

# Table
result = data[n.keys()]
data2 = result.iloc[-6:].iloc[::-1].T

cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data2.columns)]

data2.columns = cols
data2['3-year'] = result.rolling(11).mean().iloc[-1].round(2)
data2['10-year'] = result.rolling(40).mean().iloc[-1].round(2)
data2.index = data2.index.map(n)
data2 = data2.applymap('{:.2f}'.format).replace('nan', '--')
data2.to_csv(data_dir / 'cab.tex', sep='&', lineterminator='\\\ ', quotechar=' ')

As of 2023 Q2, the US runs a current account deficit of 3.1 percent of GDP, primarily as the result of a trade deficit on goods of 4.0 percent of GDP. The initial GDP report for 2023 Q3 does not include the data needed to calculate the current account balance, however, the goods trade deficit for 2023 Q3 is equivalent to 3.9 percent of GDP. 


In [69]:
#nipa_series_codes(retrieve_table('T20100'))

### Corporate Profits Destination

In [66]:
s = ['A032RC', 'A438RC', 'A054RC', 'B056RC', 'A127RC']
df = (nipa_df(retrieve_table('T11200')['Data'], s)
         / 1_000_000).dropna().rename({})
df['NNI'] = df['A032RC'] - df['A438RC']
df['TOT'] = df[['A054RC', 'B056RC', 'A127RC']].sum(axis=1)
dt = df.index[-1]
ltdate = dtxt(df.index[-1])['qtr2']
sh = df.div(df.NNI, axis=0) * 100
(sh.loc['1989':, ['A054RC', 'B056RC', 'A127RC']]
   .to_csv(data_dir / 'cprof.csv', index_label='date'))
taxrt = ((df.A054RC / df.TOT) * 100).loc['1989':]
taxrt.to_csv(data_dir / 'cprof_taxrt.csv', index_label='date')
totsh19 = sh.loc['2019', 'TOT'].mean()
divsh19 = sh.loc['2019', 'B056RC'].mean()
resh19 = sh.loc['2019', 'A127RC'].mean()
taxsh19 = sh.loc['2019', 'A054RC'].mean()
taxrt19 = taxrt.loc['2019'].mean()
text = (f'In {ltdate}, corporate profits were '+
        f'\${df.TOT.iloc[-1]:.2f} trillion, equivalent to '+
        f'{sh.TOT.iloc[-1]:.1f} percent of the income paid '+
        'to US nationals after depreciation costs (net national '+
        f'income). Of this, \${df.B056RC.iloc[-1]:.2f} trillion, '+
        f'equivalent to {sh.B056RC.iloc[-1]:.1f} percent of '+
        'net national income, were paid out as dividends '+
        '(see\cbox{blue!70!purple}), '+
        f'\${df.A127RC.iloc[-1] * 1000:,.0f} billion were '+
        'retained (corporate saving, see\cbox{cyan!50!white}), '+
        f'and \${df.A054RC.iloc[-1] * 1000:.0f} billion, '+
        f'{taxrt.iloc[-1]:.1f} percent of corporate profits, '+
        'went to corporate income tax (see\cbox{red!80!orange}). \n\n'+
        f'In 2019, corporate profits were {totsh19:.1f} percent '+
        'of net national income. Dividends were equivalent to '+
        f'{divsh19:.1f} percent, corporate savings were '+
        f'{resh19:.1f} percent, and corporate income taxes were '+
        f'{taxsh19:.1f} percent of net national income '+
        f'and {taxrt19:.1f} percent of corporate profits.')
write_txt(text_dir / 'cprof.txt', text)
print(text)

cols = ['B056RC', 'A127RC', 'A054RC']
sdf = sh[cols].iloc[-1]
height = ((sdf.cumsum() - (sdf / 2) + 1.0)).to_dict()
val = sdf.to_dict()
dtp = dtxt(sh.index[-1] + pd.DateOffset(months=3))['datetime']
nodes = [f'\\absnode{{{{{dtp}}}}}{{{height[i]}}}{{\scriptsize {val[i]:.1f}}}' 
         for i in cols]
dtv = dtxt(sh.index[-1])['qtr4'].replace(' ', '\\\\ \scriptsize ')
dtn = f'\\absnode{{{{{dtp}}}}}{{{sdf.cumsum().iloc[-1] + 3.0}}}{{\scriptsize{dtv}:}}'
nodes.append(dtn)
nodetext = '\n'.join(nodes)
write_txt(text_dir / 'cprof_nodes.txt', nodetext)

In the second quarter of 2023, corporate profits were \$3.17 trillion, equivalent to 16.0 percent of the income paid to US nationals after depreciation costs (net national income). Of this, \$1.86 trillion, equivalent to 9.4 percent of net national income, were paid out as dividends (see\cbox{blue!70!purple}), \$746 billion were retained (corporate saving, see\cbox{cyan!50!white}), and \$570 billion, 18.0 percent of corporate profits, went to corporate income tax (see\cbox{red!80!orange}). 

In 2019, corporate profits were 15.1 percent of net national income. Dividends were equivalent to 8.6 percent, corporate savings were 4.6 percent, and corporate income taxes were 1.8 percent of net national income and 12.0 percent of corporate profits.


### Corporate profits source

In [67]:
# Based loosely on Levy's Where do Profits Come From?
s = ['W170RC', 'A262RC', 'W986RC', 'A922RC']
df1 = nipa_df(retrieve_table('T50100')['Data'], s)

s = ['A123RC']
df2 = nipa_df(retrieve_table('T40100')['Data'], s)

s = ['A001RC']
df3 = nipa_df(retrieve_table('T10705')['Data'], s)

cprof = pd.DataFrame()
cprof['ROW Saving'] = (df2['A123RC'] / df3['A001RC']) * 100
cprof['HH Saving'] = (- df1['W986RC'] / df3['A001RC']) * 100
cprof['Gov Saving'] = (- df1['A922RC'] / df3['A001RC']) * 100
cprof['Investment'] = ((df1['W170RC'] - df1['A262RC']) / df3['A001RC']) * 100

cprof.loc['1989':].to_csv(data_dir / 'cprof2.csv', index_label='date')

In [72]:
#nipa_series_codes(retrieve_table('T50100'))

### Table 1.15 Prices and Costs

In [2]:
d = nipa_series_codes(retrieve_table('T11500'))
df = nipa_df(retrieve_table('T11500')['Data'], d.keys())
res = growth_contrib_ann(df, 'A455RD')[['A460RD', 'A467RD', 'A463RD']]